In [ ]:
# this file is try to cross validate between data sources

<h1>Load all dependencies</h1>

In [3]:
import os, ast, json, uuid, re
from itertools import combinations
from scipy import stats
import numpy as np
import pandas as pd
import geopandas

import lxml
from lxml import etree
from lxml.etree import QName
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.colors import n_colors
import plotly.io as pio
pio.renderers.default = "browser"
print(pio.renderers)

Renderers configuration
-----------------------
    Default renderer: 'browser'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [5]:
# optional for export packages
!pip freeze

certifi==2025.8.3
et_xmlfile==2.0.0
geopandas==1.1.1
lxml==6.0.2
numpy==2.2.3
openpyxl==3.1.5
orjson==3.11.2
packaging==25.0
pandas==2.2.3
pyarrow==19.0.1
pyogrio==0.11.1
pyproj==3.7.2
python-dateutil==2.9.0.post0
pytz==2025.1
shapely==2.1.1
six==1.17.0
tzdata==2025.1


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
dist_csv_path = 'C:/Users/user/Downloads/dist-rep-wran-ampcode-v4 (1).csv'
ws_csv_path = 'C:/Users/user/Downloads/ws_pre_license.csv'

In [11]:
xml_path = 'D:/Onedrive/SynMAR2020/แบ่งเขต.xml'
dist_csv_path = "C:/Users/ASUS/Downloads/dist-rep-wran-ampcode-v4.csv"
ws_csv_path = "C:/Users/ASUS/Downloads/ws_pre_license.csv"
sale_report_path = "D:/Onedrive/SynMAR2020/GitHub/nct_dist/nct_distr/data_source/SaleReport_redis-2025-07-31 - Sheet1 (1).csv"

In [18]:
namespaces = {
    'ss': 'urn:schemas-microsoft-com:office:spreadsheet',
    'x': 'urn:schemas-microsoft-com:office:excel',
    'o': 'urn:schemas-microsoft-com:office:office',
    'html': 'http://www.w3.org/TR/REC-html40',
    '': 'urn:schemas-microsoft-com:office:spreadsheet' # Default namespace
}

In [4]:
dist_df = pd.read_csv(dist_csv_path)

In [5]:
dist_df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1, inplace=True)

In [5]:
dist_df

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code,report_ce_year,dn_internal,amp_code2,strength,nominal_wt
0,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,2024-03-16,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205,2024-01-31,ศรีประจันต์,7205,0.25,125.00
1,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,2024-03-14,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302,2024-01-31,กะทู้,8302,0.50,0.00
2,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2024-02-28,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005,2024-01-31,บ้านโป่ง,7005,0.50,306.00
3,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104,2024-01-31,พระประแดง,1104,0.25,116.25
4,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104,2024-01-31,พระประแดง,1104,0.50,478.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,2567.0,12,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,2025-01-10,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601,2024-12-31,เมืองนราธิวาส,9601,1.00,342.00
28175,2567.0,12,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0

In [6]:
ws_df = pd.read_csv(ws_csv_path)

In [ ]:
# before cleanse

In [7]:
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885 entries, 0 to 3884
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     3885 non-null   object 
 1   ชื่อสถานที่          3885 non-null   object 
 2   ที่อยู่สถานที่       3885 non-null   object 
 3   บ้านเลขที่           3872 non-null   object 
 4   ห้อง                 281 non-null    object 
 5   ชั้น                 957 non-null    object 
 6   อาคาร                531 non-null    object 
 7   ซอย                  1314 non-null   object 
 8   ถนน                  3188 non-null   object 
 9   หมู่                 529 non-null    object 
 10  ตำบล                 3881 non-null   object 
 11  อำเภอ                3881 non-null   object 
 12  จังหวัด              3881 non-null   object 
 13  รหสไปรษณีย์          3881 non-null   float64
 14  เบอร์โทรศัพท์        3807 non-null   object 
 15  เบอร์โทรสาร          642 non-null    o

<h1>Parsing XML in case parsed csv is being used skip this part</h1>

In [62]:
ws_df

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
0,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บ้านเลขที่ 598 ชั้น 12 อาคารคิวเฮ้าส์ เพลินจิ...,598,NaN,ชั้น 12,อาคารคิวเฮ้าส์ เพลินจิต,NaN,เพลินจิต,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2617 5000,0 2627 3251,นางสาว นลิน เจนวิทย์วิชัยกุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,0.0,NaN,อนุมัติ,E
1,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บ้านเลขที่ 598 ชั้น 12 อาคารคิวเฮ้าส์ เพลินจิ...,598,NaN,ชั้น 12,อาคารคิวเฮ้าส์ เพลินจิต,NaN,เพลินจิต,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2617 5000,0 2627 3251,นางสาว นลิน เจนวิทย์วิชัยกุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,0.0,NaN,อนุมัติ,E
2,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บ้านเลขที่ 598 ชั้น 12 อาคารคิวเฮ้าส์ เพลินจิ...,598,NaN,ชั้น 12,อาคารคิวเฮ้าส์ เพลินจิต,NaN,เพลินจิต,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2617 5000,0 2627 3251,นางสาว นลิน เจนวิทย์วิชัยกุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,0.0,NaN,อนุมัติ,E
3,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ3 สถานพยาบาล เอกชน,5902038.0,283/2560 (วจ.3),อนุมัติ,NaN
4,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ4 สถานพยาบาล เอกชน,5902037.0,435/2560 (วจ.4),อนุมัติ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,นางสาว กาญจน์กมล ชามะรัตน์,ลักซ์ บางกอก คลินิกเวชกรรม,บ้านเลขที่236/2 ชั้น 1 ถนนนาคนิวาส ตำบลลาด...,236/2,NaN,ชั้น 1,NaN,NaN,นาคนิวาส,NaN,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,10230.0,0 2000 3732,NaN,นางสาว กาญจน์กมล ชามะรัตน์,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800068.0,ควจ4-ร 18/2568,อนุมัติ,NaN
3881,นาย อภิวัฒน์ บรรเทา,นิวยูคลินิกเฉพาะทางด้านเวชกรรมจิตเวช,บ้านเลขที่1105 ห้องเลขที่ R.9 ชั้น 2 โครงการ S...,1105,ห้องเลขที่ R.9,ชั้น 2,โครงการ S PLACE COMMUNITY MALL อาคาร 3,NaN,พัฒนาการ,NaN,ประเวศ,ประเวศ,กรุงเทพมหานคร,10250.0,NaN,NaN,นาย อภิวัฒน์ บรรเทา,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800069.0,ควจ4-ร 19/2568,อนุมัติ,NaN
3882,บริษัท โรช ไทยแลนด์ จำกัด,บริษัท โรชไทยแลนด์ จำกัด,บ้านเลขที่89 ชั้น 26 อาคารเอไอเอ แคปปิตอล เซ็...,89,NaN,ชั้น 26,อาคารเอไอเอ แคปปิตอล เซ็นเตอร์,NaN,รัชดาภิเษก,NaN,ดินแดง,ดินแดง,กรุงเทพมหานคร,10400.0,0 2017 5600,NaN,นาย ภูมินทร์ ยั่งยืนนาน,ครอบครอง วจ 4 เพื่อการวิเคราะห์หรือการศึกษาหรื...,6800071.0,ควจ4-ว 1/2568,อนุมัติ,NaN
3883,นาย วิษณุ เจริญสุวรรณ,แพทย์วิษณุคลินิกเวชกรรม,บ้านเลขที่427/22-23 ชั้น 1 ถนนประชาราษฎร์ 2...,427/22-23,NaN,ชั้น 1,NaN,NaN,ประชาราษฎร์ 2,NaN,บางซื่อ,บางซื่อ,กรุงเทพมหานคร,10800.0,08 3747 9889,NaN,นาย วิษณุ เจริญสุวรรณ,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800079.0,ควจ4-ร 23/2568,อนุมัติ,NaN


<h1> Parsing XML (skip this part, if using csv)</h1>

In [18]:
namespaces = {
    'ss': 'urn:schemas-microsoft-com:office:spreadsheet',
    'x': 'urn:schemas-microsoft-com:office:excel',
    'o': 'urn:schemas-microsoft-com:office:office',
    'html': 'http://www.w3.org/TR/REC-html40',
    '': 'urn:schemas-microsoft-com:office:spreadsheet' # Default namespace
}

In [7]:
tree = etree.parse(xml_path)

In [8]:
root = tree.getroot()

In [12]:
root.findall('.//ss:Worksheet', namespaces)

[<Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a2d6940>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cb80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cdc0>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16ce40>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cd80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16ca80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16da40>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16d240>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cec0>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cf80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16c800>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet

In [19]:

ss_name_qn = QName(namespaces['ss'], 'Name')
ss_row_qn = QName(namespaces['ss'], 'Row')
ss_cell_qn = QName(namespaces['ss'], 'Cell')
ss_data_qn = QName(namespaces['ss'], 'Data')
ss_type_qn = QName(namespaces['ss'], 'Type')

In [16]:
worksheet_names = []
for worksheet in root.findall('.//ss:Worksheet', namespaces):
    name = worksheet.get(ss_name)
    if name:
        worksheet_names.append(name)

In [21]:
namespaces

{'ss': 'urn:schemas-microsoft-com:office:spreadsheet',
 'x': 'urn:schemas-microsoft-com:office:excel',
 'o': 'urn:schemas-microsoft-com:office:office',
 'html': 'http://www.w3.org/TR/REC-html40',
 '': 'urn:schemas-microsoft-com:office:spreadsheet'}

In [25]:
len(root.findall('.//ss:Worksheet', namespaces)[0].find('.//ss:Table', namespaces).findall(ss_row_qn))

3886

In [26]:
first_ws_allrows = root.findall('.//ss:Worksheet', namespaces)[0].find('.//ss:Table', namespaces).findall(ss_row_qn)

In [27]:
table_data = []
for row in first_ws_allrows:
    row_data = []
    # Iterate through all Cell elements in the Row
    for cell in row.findall(ss_cell_qn):
        # Find the Data element within the Cell
        data_element = cell.find(ss_data_qn)
        if data_element is not None:
            # Get the data type and text content
            # The get() method returns a Unicode string, which is what we want
            data_value = data_element.text
            row_data.append(data_value)
        else:
            # Append an empty string or None for empty cells
            row_data.append(None) 
    table_data.append(row_data)

In [29]:
# Create the pandas DataFrame from the extracted data
df = pd.DataFrame(table_data)

# The first row often contains headers. You can set them as column names.
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

<h1>EDA starts here</h1>

In [8]:
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885 entries, 0 to 3884
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     3885 non-null   object 
 1   ชื่อสถานที่          3885 non-null   object 
 2   ที่อยู่สถานที่       3885 non-null   object 
 3   บ้านเลขที่           3872 non-null   object 
 4   ห้อง                 281 non-null    object 
 5   ชั้น                 957 non-null    object 
 6   อาคาร                531 non-null    object 
 7   ซอย                  1314 non-null   object 
 8   ถนน                  3188 non-null   object 
 9   หมู่                 529 non-null    object 
 10  ตำบล                 3881 non-null   object 
 11  อำเภอ                3881 non-null   object 
 12  จังหวัด              3881 non-null   object 
 13  รหสไปรษณีย์          3881 non-null   float64
 14  เบอร์โทรศัพท์        3807 non-null   object 
 15  เบอร์โทรสาร          642 non-null    o

In [10]:
ws_df['ประเภทคำขอ'].value_counts().reset_index()

,ประเภทคำขอ,count
0,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,759
1,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,750
2,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,542
3,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,359
4,ครอบครอง วจ 3 เพื่อการบำบัดรักษาหรือป้องกันโรค...,327
5,ครอบครอง วจ2 สถานพยาบาล เอกชน,149
6,ครอบครอง วจ4 สถานพยาบาล เอกชน,131
7,ครอบครอง วจ3 สถานพยาบาล เอกชน,100
8,นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ใ...,96
9,ครอบครอง วจ 1 เพื่อการวิเคราะห์หรือการศึกษาหรื...,74


In [9]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('ขาย')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-นิติบุคคล)                542
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-บุคคลธรรมดา)              359
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (รัฐ)                             62
ขาย วจ 1 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์      1
Name: count, dtype: int64

In [10]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('จำหน่าย')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว์ป่วยในทางการแพทย์ (หมวด ค.)        759
จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว์ป่วยในทางการแพทย์ (หมวด ก.)         60
จำหน่าย  ยส 2 เพื่อใช้ในการขนส่งสาธารณะ (หมวด ข.)                                  15
จำหน่าย ยส 2 เพื่อการศึกษาวิจัยทางด้านการแพทย์หรือทางด้านวิทยาศาสตร์ (หมวด ก.)      1
Name: count, dtype: int64

In [17]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('วจ2')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ครอบครอง วจ2 สถานพยาบาล เอกชน          147
ครอบครอง วจ2 ไม่ใช่สถานพยาบาล เอกชน      4
ครอบครอง วจ2 เพื่อใช้ทางวิทยาศาสตร์      1
Name: count, dtype: int64

In [16]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('วจ2')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ครอบครอง วจ2 สถานพยาบาล เอกชน          149
ครอบครอง วจ2 ไม่ใช่สถานพยาบาล เอกชน      4
ครอบครอง วจ2 เพื่อใช้ทางวิทยาศาสตร์      1
Name: count, dtype: int64

In [12]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('วัตถุออกฤท')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-นิติบุคคล)      542
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-บุคคลธรรมดา)    359
นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์      96
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (รัฐ)                   62
ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์        18
ครอบครอง วจ 2 เพื่อป้องกันและปราบปรามการกระทำความผิดเกี่ยวกับวัตถุออกฤทธิ์        2
ส่งออก วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์       1
Name: count, dtype: int64

In [13]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('ผลิต')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
เพื่อผลิต ยส 2                                                                             52
ครองครอง ยส 2 เพื่อผลิต ยส3 (หมวด ค.)                                                      47
ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์                  18
ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์    12
ครองครอง ยส 2 เพื่อผลิต ยส3 (หมวด ก.)                                                       2
Name: count, dtype: int64

In [15]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('ครอบครอง')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรคให้แก้ป่วยหรือสัตว์ป่วยในทางการแพทย์                                                           750
ครอบครอง วจ 3 เพื่อการบำบัดรักษาหรือป้องกันโรคให้แก้ป่วยหรือสัตว์ป่วยในทางการแพทย์                                                           327
ครอบครอง วจ2 สถานพยาบาล เอกชน                                                                                                                149
ครอบครอง วจ4 สถานพยาบาล เอกชน                                                                                                                131
ครอบครอง วจ3 สถานพยาบาล เอกชน                                                                                                                100
ครอบครอง วจ 1 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์                                                   74
ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์                            

In [14]:
ws_df['จังหวัด'].value_counts().reset_index()

,จังหวัด,count
0,กรุงเทพมหานคร,3477
1,สมุทรปราการ,74
2,ปทุมธานี,63
3,นนทบุรี,60
4,เชียงใหม่,28
5,ชลบุรี,20
6,สมุทรสาคร,14
7,นครปฐม,12
8,สงขลา,11
9,นครราชสีมา,11


In [54]:
df['เลขใบอนุญาต'].value_counts().reset_index().sort_values(by=['count'],ascending=False)

,เลขใบอนุญาต,count
0,2/2564,5
1,1/2564,5
2,4/2564,4
3,5/2564,4
4,6/2564,4
...,...,...
3358,ควจ4-ร 19/2568,1
3359,ควจ4-ว 1/2568,1
3360,ควจ4-ร 23/2568,1
3361,ควจ4-ร 21/2568,1


In [57]:
df['เลขรับ'].value_counts().reset_index().sort_values(by=['count'],ascending=False)

,เลขรับ,count
0,0,240
7,6800077,1
3608,5900758,1
23,5902038,1
3624,5902037,1
...,...,...
19,6800017,1
20,6800024,1
21,6800025,1
22,6800030,1


In [56]:
df.loc[df['เลขใบอนุญาต'] == '1/2564']

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
1817,นางสาว พรพิศ นคเรศไอศูรย์,เมดพลัสคลินิกเวชกรรม,บ้านเลขที่900/31 ห้องเลขที่ี G5 ชั้น G อาคารเอ...,900/31,ห้องเลขที่ี G5,ชั้น G,อาคารเอสวีซิตี้ อาคาร 12,None,พระราม 3,None,บางโพงพาง,ยานนาวา,กรุงเทพมหานคร,10120,06 4556 5424,None,นางสาว พรพิศ นคเรศไอศูรย์,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6400020,1/2564,อนุมัติ,None
1847,บริษัท โรงพยาบาล ซีจีเอช ลำลูกกา จำกัด,โรงพยาบาลซีจีเอช ลำลูกกา โรงพยาบาลทั่วไปขนาดใหญ่,บ้านเลขที่80/77-80 ถนน หมู่5 ตำบลลำลูกกา อ...,80/77-80,None,None,None,None,None,5,ลำลูกกา,ลำลูกกา,ปทุมธานี,12150,02 088 0888,02 088 0899,พลอากาศโท วันชัย ศิริเสรีวรรณ,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,6400392,1/2564,อนุมัติ,None
1979,บริษัท พีลาทัส มารีน จำกัด,บริษัท พีลาทัส มารีน จำกัด,บ้านเลขที่84/1-4 ชั้น 1-4 อาคารยังเพลส แกรนด์...,84/1-4,None,ชั้น 1-4,อาคารยังเพลส แกรนด์ เลอ จาแด๊งน์,พหลโยธิน 37,None,None,ลาดยาว,จตุจักร,กรุงเทพมหานคร,10900,0 2930 1801-9,None,นาย วราวิช ฉิมตะวัน,จำหน่าย ยส 2 เพื่อใช้ในการขนส่งสาธารณะ (หมวด ข.),6400812,1/2564,อนุมัติ,None
2004,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บ้านเลขที่127 ห้อง 25.02 ชั้น 25 อาคาร เกษรทาว...,127,ห้อง 25.02,ชั้น 25,อาคาร เกษรทาวเวอร์,None,ราชดำริ,None,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330,0 2073 0224,None,นางสาว สุทัชชา อัจฉริยวงศ์กุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,6400866,1/2564,อนุมัติ,None
2010,องค์การเภสัชกรรม,องค์การเภสัชกรรม,บ้านเลขที่ 75/1 ซอย ถนนพระราม 6 หมู่ ตำบลทุ...,75/1,None,None,None,None,พระราม 6,None,ทุ่งพญาไท,ราชเทวี,กรุงเทพมหานคร,10400,0 2203 8600-3,None,นาย วิฑูรย์ ด่านวิบูลย์,เพื่อผลิต ยส 2,6400868,1/2564,อนุมัติ,None


In [63]:
dist_df

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code
0,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,3/16/2024,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205
1,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,3/14/2024,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302
2,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2/28/2024,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005
3,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
4,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601
28175,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxsz

In [77]:
dist_df['CustomerAddressProvinceName'].value_counts()

CustomerAddressProvinceName
กรุงเทพมหานคร    9243
ชลบุรี           1271
นนทบุรี          1208
เชียงใหม่        1062
สมุทรปราการ       874
                 ... 
สุโขทัย            48
ยโสธร              36
มหาสารคาม          35
กาฬสินธุ์          32
สตูล               12
Name: count, Length: 76, dtype: int64

In [79]:
loc_ref_pvhos_bkk

23                                แอ็บส์เมดิคคลินิกเวชกรรม
30       เพื่อนใจคลินิกเฉพาะทางด้านเวชกรรมจิตเวชเด็กและ...
44                               สหแพทย์รัชดาคลินิกเวชกรรม
58                                       มณฑลคลินิกเวชกรรม
65                              คลินิกเวชกรรมแพทย์ลาดพร้าว
                               ...                        
26138                                    ปีติคลินิกเวชกรรม
26262                                    ไวทัลไลฟ์สหคลินิก
26487                          คลินิกเวชกรรมดุสิตาการแพทย์
26840              บ้านสาทรคลินิกเฉพาะทางด้านเวชกรรมจิตเวช
27443               ดีจีบีสหคลินิก (เวชกรรมและกายภาพบำบัด)
Name: ReferCustomerName, Length: 226, dtype: object

In [69]:
loc_ref_pvhos

1                        คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช
5                           คลินิกเวชกรรมแพทย์หญิงกนกนันท์
6                                    อนุรักษ์คลินิกเวชกรรม
17                                     สีชมพูคลินิกเวชกรรม
21       เอส ไมนด์ คลินิกเฉพาะทางด้านเวชกรรมสาขาจิตเวชศ...
                               ...                        
26840              บ้านสาทรคลินิกเฉพาะทางด้านเวชกรรมจิตเวช
26898           คลินิกเฉพาะทางด้านเวชกรรมจิตเวชแฮปปี้มายด์
27443               ดีจีบีสหคลินิก (เวชกรรมและกายภาพบำบัด)
28028                             คลินิกเวชกรรมดิเรก-พรรณี
28172                                   หมอกิ๊ฟลันสหคลินิก
Name: ReferCustomerName, Length: 611, dtype: object

In [72]:
ref_pvhos_df = pd.DataFrame(loc_ref_pvhos)

In [81]:
ref_pvhos_bkk_df = pd.DataFrame(loc_ref_pvhos_bkk)

In [73]:
ref_pvhos_df['isPresent'] = isPresent

In [82]:
ref_pvhos_bkk_df['isPresent'] = isPresent

In [85]:
ref_pvhos_bkk_df.loc[ref_pvhos_bkk_df['isPresent'] == False].loc[599]

ReferCustomerName    ่คลินิกเวชกรรมแพทย์อิทธิพล
isPresent                                 False
Name: 599, dtype: object

In [86]:
df.loc[df['ชื่อสถานที่'].str.contains('แพทย์อิทธิพล')]

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
914,นาย อิทธิพล มกรเสน,คลินิกเวชกรรมแพทย์อิทธิพล,บ้านเลขที่519/17 ซอย ถนนประชาอุทิศ หมู่ ตำบ...,519/17,None,None,None,None,ประชาอุทิศ,None,ทุ่งครุ,ทุ่งครุ,กรุงเทพมหานคร,10140,"0 2815 8825, 0 2815 9416",None,นาย อิทธิพล มกรเสน,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6100776,625/2561,อนุมัติ,None
1919,นาย อิทธิพล มกรเสน,คลินิกเวชกรรมแพทย์อิทธิพล,บ้านเลขที่519/17 ซอย ถนนประชาอุทิศ หมู่ ตำบ...,519/17,None,None,None,None,ประชาอุทิศ,None,ทุ่งครุ,ทุ่งครุ,กรุงเทพมหานคร,10140,"0 2815 8825, 0 2815 9416",None,นาย อิทธิพล มกรเสน,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6400652,ควจ4-ร 14/2564,อนุมัติ,None


In [87]:
df.to_csv("C:/Users/ASUS/Downloads/ws_pre_license.csv", index=False)

<h1>Cleansing steps</h1>

<h2>drop duplicates records</h2>

In [ ]:
# before # 3885 records

In [8]:
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885 entries, 0 to 3884
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     3885 non-null   object 
 1   ชื่อสถานที่          3885 non-null   object 
 2   ที่อยู่สถานที่       3885 non-null   object 
 3   บ้านเลขที่           3872 non-null   object 
 4   ห้อง                 281 non-null    object 
 5   ชั้น                 957 non-null    object 
 6   อาคาร                531 non-null    object 
 7   ซอย                  1314 non-null   object 
 8   ถนน                  3188 non-null   object 
 9   หมู่                 529 non-null    object 
 10  ตำบล                 3881 non-null   object 
 11  อำเภอ                3881 non-null   object 
 12  จังหวัด              3881 non-null   object 
 13  รหสไปรษณีย์          3881 non-null   float64
 14  เบอร์โทรศัพท์        3807 non-null   object 
 15  เบอร์โทรสาร          642 non-null    o

In [9]:
ws_df = ws_df.drop_duplicates()

In [8]:
# after
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     3810 non-null   object 
 1   ชื่อสถานที่          3810 non-null   object 
 2   ที่อยู่สถานที่       3810 non-null   object 
 3   บ้านเลขที่           3797 non-null   object 
 4   ห้อง                 281 non-null    object 
 5   ชั้น                 942 non-null    object 
 6   อาคาร                522 non-null    object 
 7   ซอย                  1293 non-null   object 
 8   ถนน                  3162 non-null   object 
 9   หมู่                 489 non-null    object 
 10  ตำบล                 3806 non-null   object 
 11  อำเภอ                3806 non-null   object 
 12  จังหวัด              3806 non-null   object 
 13  รหสไปรษณีย์          3806 non-null   float64
 14  เบอร์โทรศัพท์        3732 non-null   object 
 15  เบอร์โทรสาร          630 non-null    o

In [ ]:
# replaced file

In [10]:
ws_df.to_csv(ws_csv_path,index=False)

<h2>filter relevant data</h2>

In [11]:
type_keywords = ['วัตถุออกฤทธิ์ในประเภท 2', 'วจ 2', 'วจ2','ครอบครอง วจ ไม่ใช่สถานพยาบาล เอกชน','ครอบครอง วจ สถานพยาบาล เอกชน']

In [12]:
wsf_df = pd.DataFrame()
for t in type_keywords:
    loc_df = ws_df.loc[ws_df['ประเภทคำขอ'].str.contains(t)]
    wsf_df = pd.concat([wsf_df,loc_df], axis=0).drop_duplicates()

In [41]:
wsf_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1344 entries, 85 to 76
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     1344 non-null   object 
 1   ชื่อสถานที่          1344 non-null   object 
 2   ที่อยู่สถานที่       1344 non-null   object 
 3   บ้านเลขที่           1340 non-null   object 
 4   ห้อง                 125 non-null    object 
 5   ชั้น                 383 non-null    object 
 6   อาคาร                229 non-null    object 
 7   ซอย                  474 non-null    object 
 8   ถนน                  1115 non-null   object 
 9   หมู่                 162 non-null    object 
 10  ตำบล                 1343 non-null   object 
 11  อำเภอ                1343 non-null   object 
 12  จังหวัด              1343 non-null   object 
 13  รหสไปรษณีย์          1343 non-null   float64
 14  เบอร์โทรศัพท์        1314 non-null   object 
 15  เบอร์โทรสาร          232 non-null    object 

In [15]:
ct_license = ws_df['เลขใบอนุญาต'].value_counts().reset_index().sort_values(by='count',ascending=False)

In [16]:
ct_license.loc[ct_license['count']>1]

,เลขใบอนุญาต,count
0,1/2564,5
1,2/2564,5
2,6/2564,4
3,3/2564,4
4,4/2564,4
...,...,...
73,89/2563,2
74,154/2561,2
75,12/2561,2
76,12/2567,2


In [18]:
ct_license['distinct_type'] = np.nan

In [17]:
ws_df.loc[ws_df['เลขใบอนุญาต'] =='1/2564']

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
1742,นางสาว พรพิศ นคเรศไอศูรย์,เมดพลัสคลินิกเวชกรรม,บ้านเลขที่900/31 ห้องเลขที่ี G5 ชั้น G อาคารเอ...,900/31,ห้องเลขที่ี G5,ชั้น G,อาคารเอสวีซิตี้ อาคาร 12,NaN,พระราม 3,NaN,บางโพงพาง,ยานนาวา,กรุงเทพมหานคร,10120.0,06 4556 5424,NaN,นางสาว พรพิศ นคเรศไอศูรย์,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6400020.0,1/2564,อนุมัติ,NaN
1772,บริษัท โรงพยาบาล ซีจีเอช ลำลูกกา จำกัด,โรงพยาบาลซีจีเอช ลำลูกกา โรงพยาบาลทั่วไปขนาดใหญ่,บ้านเลขที่80/77-80 ถนน หมู่5 ตำบลลำลูกกา อ...,80/77-80,NaN,NaN,NaN,NaN,NaN,5,ลำลูกกา,ลำลูกกา,ปทุมธานี,12150.0,02 088 0888,02 088 0899,พลอากาศโท วันชัย ศิริเสรีวรรณ,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,6400392.0,1/2564,อนุมัติ,NaN
1904,บริษัท พีลาทัส มารีน จำกัด,บริษัท พีลาทัส มารีน จำกัด,บ้านเลขที่84/1-4 ชั้น 1-4 อาคารยังเพลส แกรนด์...,84/1-4,NaN,ชั้น 1-4,อาคารยังเพลส แกรนด์ เลอ จาแด๊งน์,พหลโยธิน 37,NaN,NaN,ลาดยาว,จตุจักร,กรุงเทพมหานคร,10900.0,0 2930 1801-9,NaN,นาย วราวิช ฉิมตะวัน,จำหน่าย ยส 2 เพื่อใช้ในการขนส่งสาธารณะ (หมวด ข.),6400812.0,1/2564,อนุมัติ,NaN
1929,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บ้านเลขที่127 ห้อง 25.02 ชั้น 25 อาคาร เกษรทาว...,127,ห้อง 25.02,ชั้น 25,อาคาร เกษรทาวเวอร์,NaN,ราชดำริ,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2073 0224,NaN,นางสาว สุทัชชา อัจฉริยวงศ์กุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,6400866.0,1/2564,อนุมัติ,NaN
1935,องค์การเภสัชกรรม,องค์การเภสัชกรรม,บ้านเลขที่ 75/1 ซอย ถนนพระราม 6 หมู่ ตำบลทุ...,75/1,NaN,NaN,NaN,NaN,พระราม 6,NaN,ทุ่งพญาไท,ราชเทวี,กรุงเทพมหานคร,10400.0,0 2203 8600-3,NaN,นาย วิฑูรย์ ด่านวิบูลย์,เพื่อผลิต ยส 2,6400868.0,1/2564,อนุมัติ,NaN


In [20]:
for i,v in ct_license.iterrows():
    license_id = v['เลขใบอนุญาต']
    ct_license.at[i, 'distinct_type'] = ws_df.loc[ws_df['เลขใบอนุญาต'] ==license_id]['ประเภทคำขอ'].nunique()

In [27]:
ct_license.loc[ct_license['count'].ne(ct_license['distinct_type'])]

,เลขใบอนุญาต,count,distinct_type


In [24]:
ct_license.loc[ct_license['count'].eq(ct_license['distinct_type'])]

,เลขใบอนุญาต,count,distinct_type
0,1/2564,5,5.0
1,2/2564,5,5.0
2,6/2564,4,4.0
3,3/2564,4,4.0
4,4/2564,4,4.0
...,...,...,...
1254,ควจ4-ร 47/2567,1,1.0
1255,จยส2-ร 40/2567,1,1.0
1256,นวจ2-บ 20/2567,1,1.0
1257,59/2567,1,1.0


In [13]:
ct_alltypes = wsf_df['ประเภทคำขอ'].value_counts().reset_index()

In [44]:
for i,v in ct_alltypes.iterrows():
    print(f"{i+1}, {v['ประเภทคำขอ']}")

1, ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-นิติบุคคล)
2, ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-บุคคลธรรมดา)
3, ครอบครอง วจ2 สถานพยาบาล เอกชน
4, นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์
5, ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (รัฐ)
6, ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์
7, นำเข้า วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์
8, ครอบครอง วจ 2 เพื่อใช้ประจำในการปฐมพยาบาลหรือในกรณีเกิดเหตุฉุกเฉินในยานพาหนะที่ใช้ในการขนส่งสาธารณะระหว่างประเทศที่จดทะเบียนในราชอาณาจักร
9, ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์
10, ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์
11, ครอบครอง วจ2 ไม่ใช่สถานพยาบาล เอกชน
12, ครอบครอง วจ 2 เพื่อป้องกันและปราบปรามการกระทำความผิดเกี่ยวกับวัตถุออกฤทธิ์
13, ครอบครอง วจ2 เพื่อใช้ทางวิทยาศาสตร์
14, ครอ

In [43]:
wsf_df['ชื่อผู้รับอนุญาต'].nunique()

959

In [46]:
wsf_df['เลขใบอนุญาต'].nunique()

1272

In [48]:
wsf_df['ที่อยู่สถานที่'].nunique()

1103

In [47]:
wsf_df['ชื่อผู้ดำเนินกิจการ'].nunique()

1105

In [49]:
ct_alltypes

,ประเภทคำขอ,count
0,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,541
1,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,358
2,ครอบครอง วจ2 สถานพยาบาล เอกชน,147
3,นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ใ...,93
4,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,62
5,ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรื...,41
6,นำเข้า วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเ...,33
7,ครอบครอง วจ 2 เพื่อใช้ประจำในการปฐมพยาบาลหรือใ...,28
8,ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในป...,18
9,ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื...,12


In [50]:
ct_alltypes['ctd_mah_name'] = np.nan
ct_alltypes['ctd_เลขใบอนุญาต'] = np.nan
ct_alltypes['ที่อยู่สถานที่'] = np.nan
ct_alltypes['ชื่อผู้ดำเนินกิจการ'] = np.nan

In [51]:
for i,v in ct_alltypes.iterrows():
    name = v['ประเภทคำขอ']
    ct_alltypes.at[i, 'ctd_mah_name'] = wsf_df.loc[wsf_df['ประเภทคำขอ'] == name]['ชื่อผู้รับอนุญาต'].nunique()
    ct_alltypes.at[i, 'ctd_เลขใบอนุญาต'] = wsf_df.loc[wsf_df['ประเภทคำขอ'] == name]['เลขใบอนุญาต'].nunique()
    ct_alltypes.at[i, 'ที่อยู่สถานที่'] = wsf_df.loc[wsf_df['ประเภทคำขอ'] == name]['ที่อยู่สถานที่'].nunique()
    ct_alltypes.at[i, 'ชื่อผู้ดำเนินกิจการ'] = wsf_df.loc[wsf_df['ประเภทคำขอ'] == name]['ชื่อผู้ดำเนินกิจการ'].nunique()

In [54]:
ct_alltypes.to_csv("C:/Users/user/Downloads/ws_pre_ctdalltypes.csv", index=False)

In [30]:
ct_alltypes['gr_types'] = np.nan

In [31]:
ct_alltypes

,ประเภทคำขอ,count,gr_types
0,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,541,NaN
1,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,358,NaN
2,ครอบครอง วจ2 สถานพยาบาล เอกชน,147,NaN
3,นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ใ...,93,NaN
4,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,62,NaN
5,ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรื...,41,NaN
6,นำเข้า วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเ...,33,NaN
7,ครอบครอง วจ 2 เพื่อใช้ประจำในการปฐมพยาบาลหรือใ...,28,NaN
8,ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในป...,18,NaN
9,ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื...,12,NaN


<h1>Cross-validate:</h1>

<h2> vs. Distribution report</h2>

In [32]:
dist_df

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code,report_ce_year,dn_internal,amp_code2,strength,nominal_wt
0,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,2024-03-16,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205,2024-01-31,ศรีประจันต์,7205,0.25,125.00
1,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,2024-03-14,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302,2024-01-31,กะทู้,8302,0.50,0.00
2,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2024-02-28,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005,2024-01-31,บ้านโป่ง,7005,0.50,306.00
3,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104,2024-01-31,พระประแดง,1104,0.25,116.25
4,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2024-02-16,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104,2024-01-31,พระประแดง,1104,0.50,478.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,2567.0,12,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,2025-01-10,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601,2024-12-31,เมืองนราธิวาส,9601,1.00,342.00
28175,2567.0,12,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0

In [33]:
loc_ref_pvhos = dist_df.loc[dist_df['CustomerTypeName'] == 'คลินิก']['ReferCustomerName'].drop_duplicates()

In [48]:
loc_ref_pvhos_bkk = dist_df.loc[(dist_df['CustomerTypeName'] == 'คลินิก') & (dist_df['CustomerAddressProvinceName'] == 'กรุงเทพมหานคร')]['ReferCustomerName'].drop_duplicates()

In [49]:
loc_ref_pvhos_bkk = loc_ref_pvhos_bkk.reset_index()

In [37]:
isPresent = []
for name in loc_ref_pvhos_bkk:
    loc_sitename = ws_df.loc[ws_df['ชื่อสถานที่'].str.contains(name,regex=False)]
    if len(loc_sitename) >0:
        isPresent.append(True)
        continue
    loc_mahname = ws_df.loc[ws_df['ชื่อผู้รับอนุญาต'].str.contains(name,regex=False)]
    if len(loc_mahname) >0:
        isPresent.append(True)
        continue
    else:
        isPresent.append(False)

In [50]:
loc_ref_pvhos_bkk['isPresent'] = isPresent

In [64]:
loc_ref_pvhos_bkk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   index              226 non-null    int64   
 1   ReferCustomerName  226 non-null    object  
 2   isPresent          226 non-null    bool    
 3   CustomerTypeName   226 non-null    object  
 4   _merge             226 non-null    category
dtypes: bool(1), category(1), int64(1), object(2)
memory usage: 6.0+ KB


<h3>Validate join</h3>

In [53]:
dist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28179 entries, 0 to 28178
Data columns (total 32 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   ReportDisburseNarcoticBeYear              28179 non-null  float64
 1   ReportDisburseNarcoticMonth               28179 non-null  int64  
 2   ProductTypeName                           28179 non-null  object 
 3   MonthlyReportNarcoticName                 28179 non-null  object 
 4   MonthlyReportNarcoticBalanceForwardValue  28179 non-null  float64
 5   MonthlyReportNarcoticReceiveValue         28179 non-null  float64
 6   MonthlyReportNarcoticPaidValue            28179 non-null  float64
 7   MonthlyReportNarcoticRemainValue          28179 non-null  float64
 8   ReferCustomerName                         28179 non-null  object 
 9   CustomerAddressProvinceName               28179 non-null  object 
 10  CustomerTypeName                  

In [54]:
hos_types_df = dist_df[['ReferCustomerName','CustomerTypeName']].drop_duplicates().copy()

In [55]:
loc_ref_pvhos_bkk = loc_ref_pvhos_bkk.merge(
    hos_types_df,
    how = 'left',
    left_on = ['ReferCustomerName'],
    right_on = ['ReferCustomerName'],
    validate = '1:1',
    indicator = True
)

In [63]:
loc_ref_pvhos_bkk.loc[loc_ref_pvhos_bkk['isPresent'] ==False].count()

index                28
ReferCustomerName    28
isPresent            28
CustomerTypeName     28
_merge               28
dtype: int64

In [65]:
loc_ref_pvhos_bkk.loc[loc_ref_pvhos_bkk['isPresent'] ==True].count()

index                198
ReferCustomerName    198
isPresent            198
CustomerTypeName     198
_merge               198
dtype: int64

In [74]:
loc_ref_pvhos_bkk.loc[loc_ref_pvhos_bkk['isPresent']==False]

,index,ReferCustomerName,isPresent,CustomerTypeName,_merge
13,234,ลาดกระบัง ซ.1 คลินิกเวชกรรม,False,คลินิก,both
18,285,คลินิกพัฒนา 25 การแพทย์เวชกรรม,False,คลินิก,both
26,405,คลินิกดุสิตาการแพทย์,False,คลินิก,both
35,494,คลินิกเวชกรรม บุญสิทธิการแพทย์,False,คลินิก,both
44,599,่คลินิกเวชกรรมแพทย์อิทธิพล,False,คลินิก,both
49,671,บลอสชั้นมคลินิกเวชกรรม,False,คลินิก,both
54,720,สหแพทย์หลังสวนสหคลินิก,False,คลินิก,both
60,798,คลินิก เวชกรรมพระราม 2,False,คลินิก,both
62,811,พาต้าโพลีคลินิกเวชกรรม,False,คลินิก,both
70,876,ทรานส์เซนต์คลินิกเวชกรรม,False,คลินิก,both


In [116]:
dist_df.loc[dist_df['ReferCustomerName'] == 'พาต้าโพลีคลินิกเวชกรรม']

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code,report_ce_year,dn_internal,amp_code2,strength,nominal_wt
811,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,29/02/2024 13:16 น.,นางสาว จริยา บัวศรี,2024-02-29,bc4c15fe-b783-4e9e-a061-f83f84d41eee,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-01-31,บางพลัด,1025,0.25,0.0
812,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,30.0,8000.0,730.0,7300.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,29/02/2024 13:16 น.,นางสาว จริยา บัวศรี,2024-02-29,969296c7-032f-4e53-b36f-0d10ed40efbc,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-01-31,บางพลัด,1025,1.00,730.0
813,2567.0,1,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,29/02/2024 13:16 น.,นางสาว จริยา บัวศรี,2024-02-29,075a18bc-a742-4460-a354-364d2f0b16e5,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-01-31,บางพลัด,1025,0.50,0.0
2712,2567.0,2,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/03/2024 9:20 น.,นางสาว จริยา บัวศรี,2024-03-27,5db6bcbd-76c5-477a-9f9f-89c3d59a1ac3,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-02-29,บางพลัด,1025,0.25,0.0
2713,2567.0,2,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,7300.0,0.0,6317.0,983.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/03/2024 9:20 น.,นางสาว จริยา บัวศรี,2024-03-27,836cfe4e-f08d-4518-b3f9-414c6d98b124,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-02-29,บางพลัด,1025,1.00,6317.0
2714,2567.0,2,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,27/03/2024 9:20 น.,นางสาว จริยา บัวศรี,2024-03-27,141dbf96-1c57-43b3-98d6-ac48792d3437,bc4c15fe-b783-4e9e-a061-f83f84d41eee,แขวงบางยี่ขัน เขตบางพลัด กรุงเทพมหานคร 10700,"Bang Yi Khan, Bang Phlat, Bangkok 10700, Thailand",APPROXIMATE,ChIJj0gCTHqZ4jARwI3iXbIAAQQ,13.773417,100.493056,"political,sublocality,sublocality_level_2",10700,บางพลัด,1025,2024-02-29,บางพลัด,1025,0.50,0.0
7075,2567.0,3,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),0.0,0.0,0.0,0.0,พาต้าโพลีคลินิกเวชกรรม,กรุงเทพมหานคร,คลินิก,RECORD,15/05/2024 13:17 น.,นาง จริยา บัวศรี,2024-05-15,2561137

In [107]:
loc_ref_pvhos_bkk.loc[216,'ReferCustomerName']

'คลินิกเวชกรรมเฉพาะทางศัลยกรรมตกแต่งและผิวหนัง เลเซอร์ พีเอไอ'

In [112]:
dist_df.loc[234]

ReportDisburseNarcoticBeYear                                                           2567.0
ReportDisburseNarcoticMonth                                                                 1
ProductTypeName                                               (วจ. 2) วัตถุออกฤทธิ์ในประเภท 2
MonthlyReportNarcoticName                   Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...
MonthlyReportNarcoticBalanceForwardValue                                                  0.0
MonthlyReportNarcoticReceiveValue                                                     10000.0
MonthlyReportNarcoticPaidValue                                                         1000.0
MonthlyReportNarcoticRemainValue                                                       9000.0
ReferCustomerName                                                 ลาดกระบัง ซ.1 คลินิกเวชกรรม
CustomerAddressProvinceName                                                     กรุงเทพมหานคร
CustomerTypeName                                            

In [117]:
ws_df[ws_df['อำเภอ'] =='บางพลัด']

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
51,นาย เกรียงไกร อุรุโสภณ,คลินิกเจตน์การแพทย์,บ้านเลขที่365 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,365,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2434 8671,NaN,นาย เกรียงไกร อุรุโสภณ,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900630.0,14/2560 (วจ.2),อนุมัติ,NaN
52,นาย เกรียงไกร อุรุโสภณ,คลินิกเวชกรรมเจตน์การแพทย์,บ้านเลขที่365 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,365,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2434 8671,NaN,นาย เกรียงไกร อุรุโสภณ,ครอบครอง วจ3 สถานพยาบาล เอกชน,5900631.0,10/2560 (วจ.3),อนุมัติ,NaN
53,นาย เกรียงไกร อุรุโสภณ,คลินิกเวชกรรมเจตน์การแพทย์,บ้านเลขที่365 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,365,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2434 8671,NaN,นาย เกรียงไกร อุรุโสภณ,ครอบครอง วจ4 สถานพยาบาล เอกชน,5900632.0,11/2560 (วจ.4),อนุมัติ,NaN
150,นาย เกตุ สายเพ็ชร์,สายเพ็ชร์คลินิกเวชกรรม,บ้านเลขที่312 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,312,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางยี่ขัน,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 6626,NaN,นาย เกตุ สายเพ็ชร์,ครอบครอง วจ2 สถานพยาบาล เอกชน,5901093.0,NaN,อนุมัติ,NaN
151,นาย เกตุ สายเพ็ชร์,สายเพ็ชร์คลินิกเวชกรรม,บ้านเลขที่312 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,312,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางยี่ขัน,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 6626,NaN,นาย เกตุ สายเพ็ชร์,ครอบครอง วจ2 สถานพยาบาล เอกชน,0.0,NaN,อนุมัติ,NaN
220,นาย สมนึก จิตสร้างบุญ,บางอ้อคลินิกเวชกรรม,บ้านเลขที่440 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,440,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางอ้อ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 9949,NaN,นาย สมนึก จิตสร้างบุญ,ครอบครอง วจ2 สถานพยาบาล เอกชน,0.0,NaN,อนุมัติ,NaN
221,นาย สมนึก จิตสร้างบุญ,บางอ้อคลินิกเวชกรรม,บ้านเลขที่440 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,440,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางอ้อ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 9949,NaN,นาย สมนึก จิตสร้างบุญ,ครอบครอง วจ4 สถานพยาบาล เอกชน,0.0,NaN,อนุมัติ,NaN
222,นาย สมนึก จิตสร้างบุญ,บางอ้อคลินิกเวชกรรม,บ้านเลขที่440 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,440,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางอ้อ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 9949,NaN,นาย สมนึก จิตสร้างบุญ,ครอบครอง วจ3 สถานพยาบาล เอกชน,5901456.0,181/2560 (วจ.3),อนุมัติ,NaN
223,นาย สมนึก จิตสร้างบุญ,บางอ้อคลินิกเวชกรรม,บ้านเลขที่440 ซอย ถนนจรัญสนิทวงศ์ หมู่ ตำบล...,440,NaN,NaN,NaN,NaN,จรัญสนิทวงศ์,NaN,บางอ้อ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2424 9949,NaN,นาย สมนึก จิตสร้างบุญ,ครอบครอง วจ4 สถานพยาบาล เอกชน,5901457.0,286/2560 (วจ.4),อนุมัติ,NaN
349,บริษัท โรงพยาบาลสัตว์ทองหล่อ จำกัด,โรงพยาบาลสัตว์ทองหล่อ สาขาปิ่นเกล้า,บ้านเลขที่493 ซอย ถนนสิรินธร หมู่3 ตำบลบา...,493,NaN,NaN,NaN,NaN,สิรินธร,3,บางบำหรุ,บางพลัด,กรุงเทพมหานคร,10700.0,0 2712 6301 ต่อ 11159,NaN,บริษัท โรงพยาบาลสัตว์ทองหล่อ จำกัด,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,0.0,NaN,อนุมัติ,NaN


In [109]:
ws_df.loc[586, 'ชื่อสถานที่']

'คลินิกเฉพาะทางด้านเวชกรรมศัลยกรรมตกแต่งและผิวหนัง เลเซอร์ พีเอไอ'

In [110]:
ws_df.loc[ws_df['ชื่อสถานที่'].str.contains('อรรถพร')]

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
678,นาย อรรถพร โสภณสฤษฎ์สุข,อรรถพรคลินิกเฉพาะทางด้านเวชกรรมศัลยกรรมตกแต่ง,บ้านเลขที่5/9 หมู่บ้านโฮมอินทาวน์ ซอย ถนนพหล...,5/9,NaN,NaN,หมู่บ้านโฮมอินทาวน์,NaN,พหลโยธิน,NaN,สนามบิน,ดอนเมือง,กรุงเทพมหานคร,10210.0,0 2996 9991-2,NaN,นาย อรรถพร โสภณสฤษฎ์สุข,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6100113.0,322/2561,อนุมัติ,NaN
1135,นาย อรรถพร โสภณสฤษฎ์สุข,อรรถพรคลินิกเฉพาะทางด้านเวชกรรมศัลยกรรมตกแต่ง,บ้านเลขที่5/9 หมู่บ้านโฮมอินทาวน์ ซอย ถนนพหล...,5/9,NaN,NaN,หมู่บ้านโฮมอินทาวน์,NaN,พหลโยธิน,NaN,สนามบิน,ดอนเมือง,กรุงเทพมหานคร,10210.0,0 2996 9991-2,NaN,นาย อรรถพร โสภณสฤษฎ์สุข,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6102637.0,ควจ4-ร 257/2562,อนุมัติ,NaN
2433,นาย อรรถพร โสภณสฤษฎ์สุข,อรรถพรคลินิกเฉพาะทางด้านเวชกรรมศัลยกรรมตกแต่ง,บ้านเลขที่5/9 หมู่บ้านโฮมอินทาวน์ ซอย ถนนพหล...,5/9,NaN,NaN,หมู่บ้านโฮมอินทาวน์,NaN,พหลโยธิน,NaN,สนามบิน,ดอนเมือง,กรุงเทพมหานคร,10210.0,0 2996 9991-2,NaN,นาย อรรถพร โสภณสฤษฎ์สุข,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,6402731.0,จยส2-ร 212/2565,อนุมัติ,E


<h2>vs. Sale report</h2>

In [12]:
# load sale report redis
salrep_df = pd.read_csv(sale_report_path)

In [16]:
salrep_df.loc[salrep_df['ProductGenericName'].str.contains('alpra', case=False)].drop_duplicates().to_csv('C:/Users/ASUS/Downloads/sale_report_alpra_all.csv',index=False)

In [17]:
salalpra_df =salrep_df.loc[salrep_df['ProductGenericName'].str.contains('alpra', case=False)].drop_duplicates()

In [19]:
salalpra_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12587 entries, 51 to 115861
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   PurchaseOrderNo                  12521 non-null  object 
 1   CustomerTypeName                 12587 non-null  object 
 2   CustomerNameTh                   12587 non-null  object 
 3   CustomerAddressProvinceName      12587 non-null  object 
 4   ProductAmount                    12587 non-null  float64
 5   PurchaseOrderItemApprovedAmount  12587 non-null  float64
 6   ProductPricePerUnit              12587 non-null  float64
 7   PurchaseOrderType                12587 non-null  object 
 8   PurchaseOrderPaymentMethodName   12587 non-null  object 
 9   ProductGenericName               12587 non-null  object 
 10  ProductTradeName                 12587 non-null  object 
 11  ReferNarcoticCategoryName        12587 non-null  object 
 12  PurchaseOrderItemAppr

In [21]:
salalpra_df['CustomerAddressProvinceName'].value_counts()

CustomerAddressProvinceName
กรุงเทพมหานคร    4868
ชลบุรี            562
นนทบุรี           535
เชียงใหม่         497
สมุทรปราการ       442
                 ... 
ตราด                7
มหาสารคาม           7
ยโสธร               6
บึงกาฬ              6
สตูล                1
Name: count, Length: 76, dtype: int64

In [33]:
salalpra_bkk_df = salalpra_df.loc[salalpra_df['CustomerAddressProvinceName'] == 'กรุงเทพมหานคร'].copy()

In [25]:
salalpra_bkk_df['CustomerNameTh'].value_counts().reset_index()

,CustomerNameTh,count
0,โรงพยาบาลสมิติเวช ศรีนครินทร์โรงพยาบาลทั่วไปขน...,69
1,โรงพยาบาลมิชชั่นโรงพยาบาลทั่วไปขนาดใหญ่,61
2,โรงพยาบาลวิชัยยุทธโรงพยาบาลทั่วไปขนาดใหญ่,60
3,โรงพยาบาลทั่วไปขนาดใหญ่พระรามเก้า,56
4,โรงพยาบาลทั่วไปขนาดใหญ่ บี.แคร์ เมดิคอลเซ็นเตอร์,55
...,...,...
392,สบายดีคลินิกเวชกรรม,1
393,ชานิสาคลินิกเวชกรรมเฉพาะทางศัลยกรรมตกแต่ง,1
394,บางเขนการแพทย์สหคลินิก,1
395,อองซองเต้ คลินิกเวชกรรม,1


In [29]:
def isPresent(left_df, right_df, left_key, right_key=[]):
    isPresent = []
    for name in left_df[left_key]:
        loc_sitename = right_df.loc[right_df[right_key[0]].str.contains(name,regex=False)]
        if len(loc_sitename) >0:
            isPresent.append(True)
            continue
        loc_mahname = right_df.loc[right_df[right_key[1]].str.contains(name,regex=False)]
        if len(loc_mahname) >0:
            isPresent.append(True)
            continue
        else:
            isPresent.append(False)
    return isPresent

In [26]:
ws_df

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
0,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บ้านเลขที่ 598 ชั้น 12 อาคารคิวเฮ้าส์ เพลินจิ...,598,NaN,ชั้น 12,อาคารคิวเฮ้าส์ เพลินจิต,NaN,เพลินจิต,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2617 5000,0 2627 3251,นางสาว นลิน เจนวิทย์วิชัยกุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,0.0,NaN,อนุมัติ,E
3,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ3 สถานพยาบาล เอกชน,5902038.0,283/2560 (วจ.3),อนุมัติ,NaN
4,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ4 สถานพยาบาล เอกชน,5902037.0,435/2560 (วจ.4),อนุมัติ,NaN
5,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ2 สถานพยาบาล เอกชน,5902039.0,497/2560 (วจ.2),อนุมัติ,NaN
6,บริษัท โรงพยาบาลผิวหนัง อโศก จำกัด,โรงพยาบาลเฉพาะทางขนาดเล็กผิวหนังอโศก สาขาปิ่นเ...,ตำบล อำเภอ จังหวัด เบอร์โทรศัพท์ เบอร์โทรสาร,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,นางสาว นันท์นภัส โปวอนุสรณ์,ครอบครอง วจ3 สถานพยาบาล เอกชน,5901045.0,104/2560 (วจ.3),อนุมัติ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,นางสาว กาญจน์กมล ชามะรัตน์,ลักซ์ บางกอก คลินิกเวชกรรม,บ้านเลขที่236/2 ชั้น 1 ถนนนาคนิวาส ตำบลลาด...,236/2,NaN,ชั้น 1,NaN,NaN,นาคนิวาส,NaN,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,10230.0,0 2000 3732,NaN,นางสาว กาญจน์กมล ชามะรัตน์,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800068.0,ควจ4-ร 18/2568,อนุมัติ,NaN
3881,นาย อภิวัฒน์ บรรเทา,นิวยูคลินิกเฉพาะทางด้านเวชกรรมจิตเวช,บ้านเลขที่1105 ห้องเลขที่ R.9 ชั้น 2 โครงการ S...,1105,ห้องเลขที่ R.9,ชั้น 2,โครงการ S PLACE COMMUNITY MALL อาคาร 3,NaN,พัฒนาการ,NaN,ประเวศ,ประเวศ,กรุงเทพมหานคร,10250.0,NaN,NaN,นาย อภิวัฒน์ บรรเทา,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800069.0,ควจ4-ร 19/2568,อนุมัติ,NaN
3882,บริษัท โรช ไทยแลนด์ จำกัด,บริษัท โรชไทยแลนด์ จำกัด,บ้านเลขที่89 ชั้น 26 อาคารเอไอเอ แคปปิตอล เซ็...,89,NaN,ชั้น 26,อาคารเอไอเอ แคปปิตอล เซ็นเตอร์,NaN,รัชดาภิเษก,NaN,ดินแดง,ดินแดง,กรุงเทพมหานคร,10400.0,0 2017 5600,NaN,นาย ภูมินทร์ ยั่งยืนนาน,ครอบครอง วจ 4 เพื่อการวิเคราะห์หรือการศึกษาหรื...,6800071.0,ควจ4-ว 1/2568,อนุมัติ,NaN
3883,นาย วิษณุ เจริญสุวรรณ,แพทย์วิษณุคลินิกเวชกรรม,บ้านเลขที่427/22-23 ชั้น 1 ถนนประชาราษฎร์ 2...,427/22-23,NaN,ชั้น 1,NaN,NaN,ประชาราษฎร์ 2,NaN,บางซื่อ,บางซื่อ,กรุงเทพมหานคร,10800.0,08 3747 9889,NaN,นาย วิษณุ เจริญสุวรรณ,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800079.0,ควจ4-ร 23/2568,อนุมัติ,NaN


In [31]:
isPresent_sale = isPresent(salalpra_bkk_df,ws_df,left_key='CustomerNameTh',right_key=['ชื่อสถานที่','ชื่อผู้รับอนุญาต'])

In [34]:
salalpra_bkk_df['isPresent_ws'] = isPresent_sale

In [35]:
salalpra_bkk_df

,PurchaseOrderNo,CustomerTypeName,CustomerNameTh,CustomerAddressProvinceName,ProductAmount,PurchaseOrderItemApprovedAmount,ProductPricePerUnit,PurchaseOrderType,PurchaseOrderPaymentMethodName,ProductGenericName,ProductTradeName,ReferNarcoticCategoryName,PurchaseOrderItemApprovedPrice,CreateDateTime,ApprovedDateTime,InvoiceNo,InvoiceDate,PurchaseStatus,สถานะคำขอซื้อ,สถานะการอนุมัติยา,gr,มูลค่า,เขต,isPresent_ws
214,NaN,สถานพยาบาลของรัฐ,โรงพยาบาล รามาธิบดี,กรุงเทพมหานคร,200.0,0.0,60.0,ONLINE,เงินเชื่อ,Alprazolam tablet 0.25 mg/tab,Alprazolam tablets 0.25 mg (Polipharm),(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,0.0,1/2/2024,NaN,NaN,NaN,7,ยกเลิก,ไม่อนุมัติ,กลุ่มยานอนหลับ,12000.0,13,True
215,NaN,สถานพยาบาลของรัฐ,โรงพยาบาล รามาธิบดี,กรุงเทพมหานคร,600.0,0.0,65.0,ONLINE,เงินเชื่อ,Alprazolam tablets 0.5 mg,Alprazolam tablets 0.5 mg (Polipharm),(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,0.0,1/2/2024,NaN,NaN,NaN,7,ยกเลิก,ไม่อนุมัติ,กลุ่มยานอนหลับ,39000.0,13,True
216,NaN,สถานพยาบาลของรัฐ,โรงพยาบาล รามาธิบดี,กรุงเทพมหานคร,200.0,0.0,70.0,ONLINE,เงินเชื่อ,Alprazolam tablet 1.0 mg,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,0.0,1/2/2024,NaN,NaN,NaN,7,ยกเลิก,ไม่อนุมัติ,กลุ่มยานอนหลับ,14000.0,13,True
15706,02-07635/2567,สถานพยาบาลของรัฐ,โรงพยาบาลเฉพาะทางศัลยกรรมตกแต่งเอเพ็กซ์ ขนาดเล็ก,กรุงเทพมหานคร,3.0,3.0,65.0,OFFLINE,เงินเชื่อ,Alprazolam tablets 0.5 mg,Alprazolam tablets 0.5 mg (Polipharm),(ยส. 2) ยาเสพติดให้โทษในประเภท 2,195.0,6/13/2024,00:00.0,NaN,NaN,7,ยกเลิก,อนุมัติ,กลุ่มยานอนหลับ,195.0,13,True
27799,03-00046/2568,สถานพยาบาลของรัฐ,โรงพยาบาล กลาง,กรุงเทพมหานคร,31.0,31.0,65.0,ONLINE,เงินเชื่อ,Alprazolam tablets 0.5 mg,Alprazolam tablets 0.5 mg (Polipharm),(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,2015.0,1/2/2025,00:00.0,ช. 68/05568,1/7/2025,4,อนุมัติ,อนุมัติ,กลุ่มยานอนหลับ,2015.0,13,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115760,03-26578/2567,คลินิก,คลินิกศิริการแพทย์,กรุงเทพมหานคร,150.0,150.0,70.0,OFFLINE,เงินสด,Alprazolam tablet 1.0 mg,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,10500.0,12/25/2024,00:00.0,ส. 68/05692,12/26/2024,4,อนุมัติ,อนุมัติ,กลุ่มยานอนหลับ,10500.0,13,True
115761,03-26578/2567,คลินิก,คลินิกศิริการแพทย์,กรุงเทพมหานคร,10.0,10.0,65.0,OFFLINE,เงินสด,Alprazolam tablets 0.5 mg,Alprazolam tablets 0.5 mg (Polipharm),(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,650.0,12/25/2024,00:00.0,ส. 68/05692,12/26/2024,4,อนุมัติ,อนุมัติ,กลุ่มยานอนหลับ,650.0,13,True
115762,03-26578/2567,คลินิก,คลินิกศิริการแพทย์,กรุงเทพมหานคร,10.0,10.0,60.0,OFFLINE,เงินสด,Alprazolam tablet 0.25 mg/tab,Alprazolam tablets 0.25 mg (Polipharm),(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,600.0,12/25/2024,00:00.0,ส. 68/05692,12/26/2024,4,อนุมัติ,อนุมัติ,กลุ่มยานอนหลับ,600.0,13,True
115774,03-26587/2567,คลินิก,คลินิกเวชกรรมโชคชัย 4 การแพทย์,กรุงเทพมหานคร,100.0,100.0,70.0,OFFLINE,เงินสด,Alprazolam tablet 1.0 mg,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,7000.0,12/25/2024,00:00.0,ส. 68/05708,12/26/2024,4,อนุมัติ,อนุมัติ,กลุ่มยานอนหลับ,7000.0,13,True


In [38]:
notfoundsale_df = salalpra_bkk_df.loc[salalpra_bkk_df['isPresent_ws'] == False].drop_duplicates(subset=['CustomerNameTh'])

In [42]:
salalpra_bkk_df['CustomerNameTh'].value_counts().info()

<class 'pandas.core.series.Series'>
Index: 397 entries, โรงพยาบาลสมิติเวช ศรีนครินทร์โรงพยาบาลทั่วไปขนาดใหญ่ to รัตตินันท์คลินิกเวชกรรม สาขาแอทเดอะฟิฟท์
Series name: count
Non-Null Count  Dtype
--------------  -----
397 non-null    int64
dtypes: int64(1)
memory usage: 6.2+ KB


In [40]:
notfoundsale_df['CustomerNameTh'].value_counts().sum()

np.int64(38)

In [157]:
notfoundsale_df['CustomerNameTh'].iloc[37]

'มอร์นิ่งมายด์คลินิกเฉพาะทางด้านเวชกรรมจิตเวช สาขาซีคอนสแควร์ ศรีนครินทร์'

In [155]:
ws_df.loc[ws_df['ชื่อสถานที่'].str.contains('นิ่งมาย')]['ชื่อสถานที่'].value_counts().reset_index().iloc[0]['ชื่อสถานที่']

'มอร์นิ่งมายด์คลินิกเฉพาะทางด้านเวชกรรมจิตเวช'

In [153]:
ws_df.loc[ws_df['ชื่อสถานที่'].str.contains('ชม')]['ชื่อสถานที่'].value_counts()

ชื่อสถานที่
คณะวิทยาศาสตร์และเทคโนโลยี มหาวิทยาลัยเทคโนโลยีราชมงคลพระนคร    1
Name: count, dtype: int64

In [46]:
notfoundsale_df.to_csv('C:/Users/ASUS/Downloads/notfoundname_salebkk.csv',index=False)

<h2>vs. Hcodes</h2>

In [158]:
hcode_csv = pd.read_csv('C:/Users/ASUS/Downloads/new_hcode - hcodes_full_2025-06-04.csv')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12400\532486495.py:1: DtypeWarning:

Columns (24,25,26,27,30) have mixed types. Specify dtype option on import or set low_memory=False.



In [160]:
hcode_csv

,#,id,name,hcode9,code9,code5,private_code,organization_type,health_office_type,health_office_type_code,organization,organization_code,department,department_code,active,address,established_date,closed_date,bed,is_client_hospital,host_agency,client_health_office_type,modified_date,notes,parsed_address,formatted_address,location_type,plcae_id,lat,lng,subtypes,pv_code,pv
0,0,1,คลินิกรักษ์ฟัน,CA0032045,3204500.0,32045.0,30103000451,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 1, 'moo': '3', 'room': null, 'alley': n...",NaN,NaN,NaN,False,NaN,NaN,2023-08-31T13:24:29.281520+07:00,[],244 ถนนมิตรภาพ ตำบล จอหอ อำเภอ เมืองนครราชสีม...,"244/3 Thanon Mittraphap, อำเภอเมือง Amphoe Mue...",ROOFTOP,ChIJSx5anrVLGTERldETCw2NG5U,15.029184,102.139203,"street_address,subpremise",30310,30
1,1,2,คลินิกนายแพทย์คธาวุฒิ,CA0032047,3204700.0,32047.0,30101000155,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 2, 'moo': '3', 'room': '', 'alley': '',...",NaN,NaN,NaN,False,NaN,NaN,2023-10-06T09:30:38.206799+07:00,[],21/10 ตำบล ขามทะเลสอ อำเภอ ขามทะเลสอ จังหวัด ...,"Kham Thale So, Kham Thale So District, Nakhon ...",APPROXIMATE,ChIJmyymJ2qxHjERUO_eyM9pBAQ,14.960898,101.969046,"locality,political",30280,30
2,2,3,คลินิกนายแพทย์เรืองศักดิ์,CA0032051,3205100.0,32051.0,30101000756,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 3, 'moo': '4', 'room': null, 'alley': n...",NaN,NaN,NaN,False,NaN,NaN,2023-08-31T13:24:29.330704+07:00,[],519 ถนนราษฎร์พัฒนา ตำบล แชะ อำเภอ ครบุรี จังห...,"Chae, Khon Buri District, Nakhon Ratchasima, T...",APPROXIMATE,ChIJ9yS64xzlGzEREOTeyM9pBAQ,14.543317,102.277684,"locality,political",30250,30
3,3,4,คลินิกทีปวิทย์การแพทย์,CA0032053,3205300.0,32053.0,30101003656,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 4, 'moo': '9', 'room': '', 'alley': '',...",NaN,NaN,NaN,False,NaN,NaN,2023-10-06T09:36:23.436897+07:00,[],543 ตำบล บ้านใหม่ อำเภอ ครบุรี จังหวัด นครราช...,"Ban Mai, Khon Buri District, Nakhon Ratchasima...",APPROXIMATE,ChIJDcoN0w3xGzERgOTeyM9pBAQ,14.443365,102.220463,"locality,political",30250,30
4,4,5,คลินิกแพทย์นิพนธ์,CA0032065,3206500.0,32065.0,-,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 5, 'moo': '0', 'room': null, 'alley': n...",NaN,NaN,NaN,False,NaN,NaN,2023-10-09T09:29:57.515119+07:00,[],805 ถนนสุรนารายณ์ ตำบล โนนไทย อำเภอ โนนไทย จั...,"805 Sura Narai Rd, Tambon Non Thai, Amphoe Non...",RANGE_INTERPOLATED,EmA4MDUgU3VyYSBOYXJhaSBSZCwgVGFtYm9uIE5vbiBUaG...,15.183961,102.076685,street_address,30220,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41636,41637,41640,คลินิกเจนจิการพยาบาลและการผดุงครรภ์,CA0000413,NaN,NaN,36105003068,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41640, 'moo': '8', 'room': '', 'alley':...",2025-03-03,NaN,0.0,False,NaN,NaN,2025-05-06T17:03:17.565266+07:00,"[{'id': 21151, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,36000,36
41637,41638,41641,วรรณิภาคลินิกการพยาบาลและการผดุงครรภ์,CA0000414,NaN,NaN,32105002968,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41641, 'moo': '9', 'room': '-', 'alley'...",2025-05-29,NaN,0.0,False,NaN,NaN,2025-05-06T17:08:00.146753+07:00,"[{'id': 21152, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,32130,32
41638,41639,41642,ศดานันท์คลินิกการพยาบาลและการผดุงครรภ์,CA0000415,NaN,NaN,33105003768,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41642, 'moo': '13', 'room': '', 'alley'...",2025-05-01,NaN,0.0,False,NaN,NaN,2025-05-06T17:13:06.244748+07:00,"[{'id': 21153, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,33350,33
41639,41640,41643,สุนาภา คลินิกการพยาบาลและการผดุงครรภ์,CA0000416,NaN,NaN,24105000268,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41643, 'moo': '10', 'room': '', 'alley'...",2025-02-13,NaN,0.0,False,NaN,NaN,2025-05-06T17:17:28.244554+07:00,"[{'id': 21154, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,24120,24


In [159]:
hcode_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41641 entries, 0 to 41640
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   #                          41641 non-null  int64  
 1   id                         41641 non-null  int64  
 2   name                       41641 non-null  object 
 3   hcode9                     41641 non-null  object 
 4   code9                      41218 non-null  float64
 5   code5                      41218 non-null  float64
 6   private_code               27071 non-null  object 
 7   organization_type          41641 non-null  object 
 8   health_office_type         41641 non-null  object 
 9   health_office_type_code    41641 non-null  int64  
 10  organization               41641 non-null  object 
 11  organization_code          41641 non-null  int64  
 12  department                 14963 non-null  object 
 13  department_code            14963 non-null  flo

In [162]:
hcode_csv.loc[hcode_csv['address'].str.contains('กรุงเทพ')]

,#,id,name,hcode9,code9,code5,private_code,organization_type,health_office_type,health_office_type_code,organization,organization_code,department,department_code,active,address,established_date,closed_date,bed,is_client_hospital,host_agency,client_health_office_type,modified_date,notes,parsed_address,formatted_address,location_type,plcae_id,lat,lng,subtypes,pv_code,pv
38,38,39,เค.พี.เอส.การแพทย์คลินิกเฉพาะทางด้านเวชกรรมอาย...,CA0041369,4136900.0,41369.0,10102003160,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 39, 'moo': '0', 'room': null, 'alley': ...",2017-07-15,NaN,NaN,False,NaN,NaN,2023-08-31T13:24:30.386329+07:00,[],981/13-15 ถนนบางขุนเทียน-ชายทะเล ตำบล แสมดำ อ...,"981 Bang Khun Thian Road, Khwaeng Samae Dam, K...",ROOFTOP,ChIJCf-cIsy84jARi73GFMnoMaU,13.631545,100.434695,"premise,street_address",10150,10
39,39,40,เอพีเอสคลินิกเฉพาะทางด้านเวชกรรมไตเทียม,CA0041370,4137000.0,41370.0,10102002560,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 40, 'moo': '0', 'room': null, 'alley': ...",2017-06-01,NaN,NaN,False,NaN,NaN,2023-08-31T13:24:30.428488+07:00,[],60 ซอยพหลโยธิน4 ถนนพหลโยธิน ตำบล สามเสนใน อำเ...,"60 Phaholyothin 4, Khwaeng Samsen Nai, Khet Ph...",ROOFTOP,ChIJle4zAh6c4jARGApYJMbLrzY,13.779841,100.546902,street_address,10400,10
252,252,253,เออีซี คลินิกเวชกรรม,CA0041381,4138100.0,41381.0,10101009060,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 253, 'moo': '0', 'room': null, 'alley':...",NaN,NaN,NaN,False,NaN,NaN,2023-08-31T13:24:36.849623+07:00,[],17/6 ซอยลาดพร้าว64 แยก 4 ตำบล วังทองหลาง อำเภ...,"17/6 Thanon Lat Phrao 64 Yaek 4, Khwaeng Wang ...",ROOFTOP,ChIJ7c5RB-id4jARKd99ezQQbDk,13.788970,100.597519,"street_address,subpremise",10310,10
345,345,346,โรงพยาบาลสวนเบญจกิติเฉลิมพระเกียรติ84พรรษา,FA0011532,1153200.0,11532.0,NaN,รัฐบาล,โรงพยาบาล นอก สธ.,12,กระทรวงการคลัง,3000,-,0.0,True,"{'id': 346, 'moo': '0', 'room': null, 'alley':...",NaN,NaN,82.0,False,NaN,NaN,2023-08-31T13:24:39.910429+07:00,"[{'id': 6320, 'name': 'เปลี่ยนชื่อหน่วยบริการส...",184 ถ.พระราม 4 ตำบล คลองเตย อำเภอ คลองเตย จัง...,"184 Kluai Nam Thai Rd, Khwaeng Phra Khanong, K...",RANGE_INTERPOLATED,EmUxODQgS2x1YWkgTmFtIFRoYWkgUmQsIEtod2FlbmcgUG...,13.708872,100.586237,street_address,10110,10
1186,1186,1187,ธราวรรณสหคลินิก,CA0021722,2172200.0,21722.0,10110000358,เอกชน,คลินิกเอกชน,16,เอกชน,90000,-,0.0,False,"{'id': 1187, 'moo': '9', 'room': null, 'alley'...",NaN,2017-08-28,NaN,False,NaN,NaN,2023-08-31T13:25:04.578290+07:00,"[{'id': 6342, 'name': 'เปลี่ยนชื่อหน่วยบริการส...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10230,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41531,41532,41535,กรุงเทพ แล็บ คลินิกเทคนิคการแพทย์,CA0000309,NaN,NaN,10107000668,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41535, 'moo': '', 'room': '', 'alley': ...",2025-04-10,NaN,0.0,False,NaN,NaN,2025-04-28T13:08:32.959679+07:00,"[{'id': 21041, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10120,10
41535,41536,41539,พาราไดซ์ คลินิกเวชกรรม,CA0000313,NaN,NaN,10101016968,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41539, 'moo': '', 'room': 'PDP.F03.CS00...",2025-04-10,NaN,0.0,False,NaN,NaN,2025-04-28T16:18:57.470409+07:00,"[{'id': 21045, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10250,10
41549,41550,41553,กรุงเทพ แล็บ คลินิกเทคนิคการแพทย์,CA0000327,NaN,NaN,10107000768,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41553, 'moo': '4', 'room': '', 'alley':...",2025-04-10,NaN,0.0,False,NaN,NaN,2025-04-29T13:28:26.245504+07:00,"[{'id': 21062, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10150,10
41575,41576,41579,เอ็มพีแคร์คลินิกกายภาพบำบัด,CA0000353,NaN,NaN,10106002566,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41579, 'moo': '', 'room': 'เลขที่ 300D'...",2023-06-08,NaN,0.0,False,NaN,NaN,2025-04-30T13:59:59.538382+07:00,"[{'id': 21092, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10110,1

In [167]:
hcode_csv.loc[hcode_csv['name'].str.contains('ลักซ์')]

,#,id,name,hcode9,code9,code5,private_code,organization_type,health_office_type,health_office_type_code,organization,organization_code,department,department_code,active,address,established_date,closed_date,bed,is_client_hospital,host_agency,client_health_office_type,modified_date,notes,parsed_address,formatted_address,location_type,plcae_id,lat,lng,subtypes,pv_code,pv
3575,3576,3577,คลินิกทันตกรรมเดนทัลดีลักซ์,CA0043050,4305000.0,43050.0,13103002265,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 3577, 'moo': '19', 'room': null, 'alley...",2022-04-01,NaN,NaN,False,NaN,NaN,2023-08-31T13:26:35.268475+07:00,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,12120,12
31200,31201,31203,คลินิกทันตกรรม เดนทัล ดีลักซ์,CA0045011,4501100.0,45011.0,14103000958,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 31203, 'moo': '5', 'room': '', 'alley':...",2015-11-19,NaN,0.0,False,NaN,NaN,2023-11-14T09:27:42.946731+07:00,"[{'id': 8012, 'name': 'กำหนดรหัสใหม่', 'detail...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13210,13
31547,31548,31550,คลินิกกายภาพบำบัดเบรนรีแฮบลักซ์,CA0045356,4535600.0,45356.0,10106003165,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 31550, 'moo': '', 'room': 'เลขที่ B-13'...",2022-04-26,NaN,0.0,False,NaN,NaN,2023-12-26T14:09:33.398528+07:00,"[{'id': 8441, 'name': 'กำหนดรหัสใหม่', 'detail...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10110,10
41476,41477,41480,คลินิกทันตกรรมลักซ์,CA0000255,NaN,NaN,10103001368,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 41480, 'moo': '-', 'room': 'เลขที่ B107...",2025-03-17,NaN,0.0,False,NaN,NaN,2025-04-24T11:01:12.203724+07:00,"[{'id': 20983, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10240,10


In [166]:
ws_df.loc[ws_df['จังหวัด'] == 'กรุงเทพมหานคร']

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
0,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บ้านเลขที่ 598 ชั้น 12 อาคารคิวเฮ้าส์ เพลินจิ...,598,NaN,ชั้น 12,อาคารคิวเฮ้าส์ เพลินจิต,NaN,เพลินจิต,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2617 5000,0 2627 3251,นางสาว นลิน เจนวิทย์วิชัยกุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,0.0,NaN,อนุมัติ,E
3,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ3 สถานพยาบาล เอกชน,5902038.0,283/2560 (วจ.3),อนุมัติ,NaN
4,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ4 สถานพยาบาล เอกชน,5902037.0,435/2560 (วจ.4),อนุมัติ,NaN
5,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ2 สถานพยาบาล เอกชน,5902039.0,497/2560 (วจ.2),อนุมัติ,NaN
8,นางสาว นันฐิณี ทิมเกลี้ยง,เดิมแคร์คลินิกเวชกรรม สาขารามอินทรา,"บ้านเลขที่ 587,589 ศูนย์การค้าเดอะพรอมานาด ชั้...","587,589 ศูนย์การค้าเดอะพรอมานาด ชั้น4 พื้นที่ห...",NaN,NaN,NaN,NaN,รามอินทรา,NaN,คันนายาว,คันนายาว,กรุงเทพมหานคร,10230.0,0 2947 5699,NaN,นางสาว นันฐิณี ทิมเกลี้ยง,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900758.0,65/2560 (วจ.2),อนุมัติ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,นางสาว กาญจน์กมล ชามะรัตน์,ลักซ์ บางกอก คลินิกเวชกรรม,บ้านเลขที่236/2 ชั้น 1 ถนนนาคนิวาส ตำบลลาด...,236/2,NaN,ชั้น 1,NaN,NaN,นาคนิวาส,NaN,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,10230.0,0 2000 3732,NaN,นางสาว กาญจน์กมล ชามะรัตน์,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800068.0,ควจ4-ร 18/2568,อนุมัติ,NaN
3881,นาย อภิวัฒน์ บรรเทา,นิวยูคลินิกเฉพาะทางด้านเวชกรรมจิตเวช,บ้านเลขที่1105 ห้องเลขที่ R.9 ชั้น 2 โครงการ S...,1105,ห้องเลขที่ R.9,ชั้น 2,โครงการ S PLACE COMMUNITY MALL อาคาร 3,NaN,พัฒนาการ,NaN,ประเวศ,ประเวศ,กรุงเทพมหานคร,10250.0,NaN,NaN,นาย อภิวัฒน์ บรรเทา,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800069.0,ควจ4-ร 19/2568,อนุมัติ,NaN
3882,บริษัท โรช ไทยแลนด์ จำกัด,บริษัท โรชไทยแลนด์ จำกัด,บ้านเลขที่89 ชั้น 26 อาคารเอไอเอ แคปปิตอล เซ็...,89,NaN,ชั้น 26,อาคารเอไอเอ แคปปิตอล เซ็นเตอร์,NaN,รัชดาภิเษก,NaN,ดินแดง,ดินแดง,กรุงเทพมหานคร,10400.0,0 2017 5600,NaN,นาย ภูมินทร์ ยั่งยืนนาน,ครอบครอง วจ 4 เพื่อการวิเคราะห์หรือการศึกษาหรื...,6800071.0,ควจ4-ว 1/2568,อนุมัติ,NaN
3883,นาย วิษณุ เจริญสุวรรณ,แพทย์วิษณุคลินิกเวชกรรม,บ้านเลขที่427/22-23 ชั้น 1 ถนนประชาราษฎร์ 2...,427/22-23,NaN,ชั้น 1,NaN,NaN,ประชาราษฎร์ 2,NaN,บางซื่อ,บางซื่อ,กรุงเทพมหานคร,10800.0,08 3747 9889,NaN,นาย วิษณุ เจริญสุวรรณ,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800079.0,ควจ4-ร 23/2568,อนุมัติ,NaN


In [168]:
wsbkk_df = ws_df.loc[ws_df['จังหวัด'] == 'กรุงเทพมหานคร']

In [169]:
hcode_df = hcode_csv[['name', 'hcode9']]

In [172]:
wsbkk_df = wsbkk_df.drop_duplicates(subset=['ชื่อสถานที่'])

In [174]:
merge = wsbkk_df.merge(
    hcode_df,
    how = 'left',
    left_on = ['ชื่อสถานที่'],
    right_on = ['name'],
    validate = '1:m',
    indicator = True
)

In [226]:
merge['_merge'].value_counts()

_merge
left_only     1043
both           288
right_only       0
Name: count, dtype: int64

In [179]:
merge.loc[merge['_merge'] == 'both']['ชื่อสถานที่'].value_counts()

ชื่อสถานที่
บ้านหมอคลินิกเวชกรรม                                        4
คลินิกทันตกรรมมีเมาธ                                        3
คลินิกแพทย์ธงชัย                                            3
คลินิกศูนย์แพทย์พัฒนา                                       2
คลินิกไพโรจน์การแพทย์                                       2
                                                           ..
ว.พ.คลินิกการแพทย์                                          1
อิสรภาพคลินิกเวชกรรม                                        1
โรงพยาบาลสมิติเวช ศรีนครินทร์โรงพยาบาลทั่วไปขนาดใหญ่        1
โรงพยาบาลทั่วไปขนาดใหญ่วิชัยเวชอินเตอร์เนชั่นแนล หนองแขม    1
สังข์กระจายคลินิกการแพทย์                                   1
Name: count, Length: 271, dtype: int64

In [180]:
hcode_df.loc[hcode_df['name'] == 'บ้านหมอคลินิกเวชกรรม']

,name,hcode9
6166,บ้านหมอคลินิกเวชกรรม,CA0024389
7531,บ้านหมอคลินิกเวชกรรม,CA0034957
33424,บ้านหมอคลินิกเวชกรรม,CA0047231
39581,บ้านหมอคลินิกเวชกรรม,CA0054438


In [184]:
hcode_csv.loc[hcode_csv['name'] == 'บ้านหมอคลินิกเวชกรรม']

,#,id,name,hcode9,code9,code5,private_code,organization_type,health_office_type,health_office_type_code,organization,organization_code,department,department_code,active,address,established_date,closed_date,bed,is_client_hospital,host_agency,client_health_office_type,modified_date,notes,parsed_address,formatted_address,location_type,plcae_id,lat,lng,subtypes,pv_code,pv
6166,6167,6168,บ้านหมอคลินิกเวชกรรม,CA0024389,2438900.0,24389.0,10101066752,เอกชน,คลินิกเอกชน,16,เอกชน,90000,-,0.0,True,"{'id': 6168, 'moo': '0', 'room': null, 'alley'...",NaN,NaN,NaN,False,NaN,NaN,2023-08-31T13:28:12.886244+07:00,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,10120,10
7531,7532,7533,บ้านหมอคลินิกเวชกรรม,CA0034957,3495700.0,34957.0,55101000157,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 7533, 'moo': '0', 'room': null, 'alley'...",NaN,NaN,NaN,False,NaN,NaN,2023-08-31T13:28:57.609231+07:00,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,55000,55
33424,33425,33428,บ้านหมอคลินิกเวชกรรม,CA0047231,4723100.0,47231.0,30101003866,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 33428, 'moo': '2', 'room': '', 'alley':...",2023-08-18,NaN,0.0,False,NaN,NaN,2024-02-13T14:01:29.012225+07:00,"[{'id': 10627, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,30140,30
39581,39582,39585,บ้านหมอคลินิกเวชกรรม,CA0054438,5443800.0,54438.0,51101000268,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 39585, 'moo': '1', 'room': '', 'alley':...",2025-01-20,NaN,0.0,False,NaN,NaN,2025-01-22T07:09:17.546856+07:00,"[{'id': 18512, 'name': 'กำหนดรหัสใหม่', 'detai...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,54000,54


In [178]:
merge.loc[merge['_merge'] == 'both'].drop_duplicates(subset=['ชื่อสถานที่'])

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ,name,hcode9,_merge
12,นาย ชวนัชย์ จรรยาศักดิ์,คลินิกหมอเด็ก,บ้านเลขที่12/12 ซอย ถนนเทศบาลสงเคราะห์ หมู่...,12/12,NaN,NaN,NaN,NaN,เทศบาลสงเคราะห์,NaN,ลาดยาว,จตุจักร,กรุงเทพมหานคร,10900.0,0 2591 5248,NaN,นาย ชวนัชย์ จรรยาศักดิ์,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900608.0,6/2560 (วจ.2),อนุมัติ,NaN,คลินิกหมอเด็ก,CA0040209,both
14,นาย อุทัย ธารธนาภรณ์,คลินิกอุทัยการแพทย์,บ้านเลขที่52/15-16 ซอยรามคำแหง 60/5 ถนนสุขา...,52/15-16,NaN,NaN,NaN,รามคำแหง 60/5,สุขาภิบาล 3,NaN,หัวหมาก,บางกะปิ,กรุงเทพมหานคร,10240.0,0 2374 8572,NaN,นาย อุทัย ธารธนาภรณ์,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900611.0,7/2560 (วจ.2),อนุมัติ,NaN,คลินิกอุทัยการแพทย์,CA0034655,both
23,นาย ทวี สมศิริวัฒนา,นครไทยคลินิกการแพทย์,บ้านเลขที่495/21 ซอย ถนนสาธุประดิษฐ์ หมู่ ต...,495/21,NaN,NaN,NaN,NaN,สาธุประดิษฐ์,NaN,ช่องนนทรี,ยานนาวา,กรุงเทพมหานคร,10120.0,0 2211 8771,NaN,นพ. ทวี สมศิริวัฒนา,ครอบครอง วจ2 สถานพยาบาล เอกชน,0.0,NaN,อนุมัติ,NaN,นครไทยคลินิกการแพทย์,CA0016289,both
28,บริษัท มีนบุรี การแพทย์ จำกัด,นวมินทร์ 9 คลินิกเวชกรรม,บ้านเลขที่92 ห้องเลขที่ A1-A15 ชั้น 2 อาคารน...,92,ห้องเลขที่ A1-A15,ชั้น 2,อาคารนวมินทร์พลาซ่า,NaN,สีหบุรานุกิจ,NaN,มีนบุรี,มีนบุรี,กรุงเทพมหานคร,10510.0,0 2518 1818,NaN,นายสมภพ เหล่าสัจจะ,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900765.0,73/2560 (วจ.2),อนุมัติ,NaN,นวมินทร์ 9 คลินิกเวชกรรม,CA0041942,both
31,นาย มาโนชญ์ ลีโทชวลิต,สังข์กระจายคลินิกการแพทย์,บ้านเลขที่849/4 ซอยวัดสังข์กระจาย ถนนเพชรเก...,849/4,NaN,NaN,NaN,วัดสังข์กระจาย,เพชรเกษม,NaN,วัดท่าพระ,บางกอกใหญ่,กรุงเทพมหานคร,10600.0,0 2466 8564,NaN,นาย มาโนชญ์ ลีโทชวลิต,ครอบครอง วจ4 สถานพยาบาล เอกชน,5900667.0,22/2560 (วจ.4),อนุมัติ,NaN,สังข์กระจายคลินิกการแพทย์,CA0024201,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,บริษัท นวศรี การแพทย์ จำกัด,โรงพยาบาลผู้ป่วยเรื้อรังศุขเวชขนาดเล็ก,บ้านเลขที่20/6 ซอยนวศรี 2 ถนนรามคำแหง 21 หม...,20/6,NaN,NaN,NaN,นวศรี 2,รามคำแหง 21,NaN,พลับพลา,วังทองหลาง,กรุงเทพมหานคร,10310.0,0 2319 5870-1,0 2319 5871,นาย มานิตย์ ประพันธ์ศิลป์,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6700649.0,121/2567,อนุมัติ,NaN,โรงพยาบาลผู้ป่วยเรื้อรังศุขเวชขนาดเล็ก,HA0014578,both
1312,บริษัท เป็นเลิศเมดิคอล จำกัด,สิทธิรักษ์คลินิกเวชกรรม,"บ้านเลขที่128,130 ชั้น 1 ถนนเลียบวารี ตำบล...","128,130",NaN,ชั้น 1,NaN,NaN,เลียบวารี,NaN,กระทุ่มราย,หนองจอก,กรุงเทพมหานคร,10530.0,0 2100 4542,NaN,นาย นันทศักดิ์ ธรรมานวัตร์,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800017.0,ควจ4-ร 6/2568,อนุมัติ,NaN,สิทธิรักษ์คลินิกเวชกรรม,CA0048975,both
1315,มูลนิธิ ฮั่วเคี้ยวป่อเต็กเซี่ยงตึ๊ง(ป่อเต็กตึ๊ง),โรงพยาบาลหัวเฉียวโรงพยาบาลทั่วไปขนาดใหญ่,บ้านเลขที่665 ซอย ถนนบำรุงเมือง หมู่ ตำบลคล...,665,NaN,NaN,NaN,NaN,บำรุงเมือง,NaN,คลองมหานาค,ป้อมปราบศัตรูพ่าย,กรุงเทพมหานคร,10100.0,0 2223 1351 ต่อ 5507-8,0 2223 1253,นาง มนนภา ขุนณรงค์,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800030.0,ควจ4-ร 9/2568,อนุมัติ,NaN,โรงพยาบาลหัวเฉียวโรงพยาบาลทั่วไปขนาดใหญ่,HA0011750,both
1329,นาย วิษณุ เจริญสุวรรณ,แพทย์วิษณุคลินิกเวชกรรม,บ้านเลขที่427/22-23 ชั้น 1 ถนนประชาราษฎร์ 2...,427/22-23,NaN,ชั้น 1,NaN,NaN,ประชาราษฎร์ 2,NaN,บางซื่อ,บางซื่อ,กรุงเทพมหานคร,10800.0,08 3747 9889,NaN,นาย วิษณุ เจริญสุวรรณ,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800079.0,ควจ4-ร 23/2568,อนุมัติ,NaN,แพทย์วิษณุคลินิกเวชกรรม,CA0056029,both


In [185]:
merge2 = merge.loc[merge['_merge'] == 'both'].drop_duplicates(subset=['ชื่อสถานที่'])

In [186]:
merge2.drop(columns=['_merge'],inplace=True)

In [189]:
merge2 = merge2.merge(
    hcode_csv[['hcode9','address']].drop_duplicates(subset=['hcode9']),
    how='left',
    left_on =['hcode9'],
    right_on = ['hcode9'],
    validate = '1:1',
    indicator = True
)

In [196]:
merge2['address'].iloc[0]

"{'id': 11043, 'moo': '9', 'room': null, 'alley': null, 'floor': null, 'steet': null, 'number': null, 'region': 'ภาคกลาง', 'details': '61/4\\t\\tถนนเทพารักษ์\\t', 'village': null, 'zipcode': '10540', 'building': null, 'district': 'บางพลี', 'province': 'สมุทรปราการ', 'health_zone': '06', 'subdistrict': 'บางพลีใหญ่', 'district_code': '1103', 'province_code': '11', 'province_type': 'จังหวัด', 'is_double_space': false, 'subdistrict_code': '110301'}"

In [197]:
processed_string = (merge2['address'].iloc[0]).replace('null', 'None')

In [198]:
processed_string = processed_string.replace('false', 'False')

In [199]:
ast.literal_eval(processed_string)

{'id': 11043,
 'moo': '9',
 'room': None,
 'alley': None,
 'floor': None,
 'steet': None,
 'number': None,
 'region': 'ภาคกลาง',
 'details': '61/4\t\tถนนเทพารักษ์\t',
 'village': None,
 'zipcode': '10540',
 'building': None,
 'district': 'บางพลี',
 'province': 'สมุทรปราการ',
 'health_zone': '06',
 'subdistrict': 'บางพลีใหญ่',
 'district_code': '1103',
 'province_code': '11',
 'province_type': 'จังหวัด',
 'is_double_space': False,
 'subdistrict_code': '110301'}

In [252]:
def parsed_address(str_obj):
    try :
        proc_str = str_obj.replace('null', 'None')
        proc_str = proc_str.replace('false', 'False')
        proc_str = proc_str.replace('true', 'True')
        return ast.literal_eval(proc_str)
    except ValueError as e:
        return None

In [201]:
merge2['parsed_address'] = merge2['address'].apply(parsed_address)

In [202]:
def get_dsitrict(obj):
    return obj['district']

In [203]:
merge2['district'] = merge2['parsed_address'].apply(get_dsitrict)

In [211]:
merge2['eq_district'] = merge2['district'].eq(merge2['อำเภอ'])

In [207]:
merge2

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ,name,hcode9,address,_merge,parsed_address,district,eq_district
0,นาย ชวนัชย์ จรรยาศักดิ์,คลินิกหมอเด็ก,บ้านเลขที่12/12 ซอย ถนนเทศบาลสงเคราะห์ หมู่...,12/12,NaN,NaN,NaN,NaN,เทศบาลสงเคราะห์,NaN,ลาดยาว,จตุจักร,กรุงเทพมหานคร,10900.0,0 2591 5248,NaN,นาย ชวนัชย์ จรรยาศักดิ์,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900608.0,6/2560 (วจ.2),อนุมัติ,NaN,คลินิกหมอเด็ก,CA0040209,"{'id': 11043, 'moo': '9', 'room': null, 'alley...",both,"{'id': 11043, 'moo': '9', 'room': None, 'alley...",บางพลี,False
1,นาย อุทัย ธารธนาภรณ์,คลินิกอุทัยการแพทย์,บ้านเลขที่52/15-16 ซอยรามคำแหง 60/5 ถนนสุขา...,52/15-16,NaN,NaN,NaN,รามคำแหง 60/5,สุขาภิบาล 3,NaN,หัวหมาก,บางกะปิ,กรุงเทพมหานคร,10240.0,0 2374 8572,NaN,นาย อุทัย ธารธนาภรณ์,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900611.0,7/2560 (วจ.2),อนุมัติ,NaN,คลินิกอุทัยการแพทย์,CA0034655,"{'id': 10945, 'moo': '0', 'room': null, 'alley...",both,"{'id': 10945, 'moo': '0', 'room': None, 'alley...",หาดใหญ่,False
2,นาย ทวี สมศิริวัฒนา,นครไทยคลินิกการแพทย์,บ้านเลขที่495/21 ซอย ถนนสาธุประดิษฐ์ หมู่ ต...,495/21,NaN,NaN,NaN,NaN,สาธุประดิษฐ์,NaN,ช่องนนทรี,ยานนาวา,กรุงเทพมหานคร,10120.0,0 2211 8771,NaN,นพ. ทวี สมศิริวัฒนา,ครอบครอง วจ2 สถานพยาบาล เอกชน,0.0,NaN,อนุมัติ,NaN,นครไทยคลินิกการแพทย์,CA0016289,"{'id': 5867, 'moo': '0', 'room': null, 'alley'...",both,"{'id': 5867, 'moo': '0', 'room': None, 'alley'...",ยานนาวา,True
3,บริษัท มีนบุรี การแพทย์ จำกัด,นวมินทร์ 9 คลินิกเวชกรรม,บ้านเลขที่92 ห้องเลขที่ A1-A15 ชั้น 2 อาคารน...,92,ห้องเลขที่ A1-A15,ชั้น 2,อาคารนวมินทร์พลาซ่า,NaN,สีหบุรานุกิจ,NaN,มีนบุรี,มีนบุรี,กรุงเทพมหานคร,10510.0,0 2518 1818,NaN,นายสมภพ เหล่าสัจจะ,ครอบครอง วจ2 สถานพยาบาล เอกชน,5900765.0,73/2560 (วจ.2),อนุมัติ,NaN,นวมินทร์ 9 คลินิกเวชกรรม,CA0041942,"{'id': 2141, 'moo': '0', 'room': null, 'alley'...",both,"{'id': 2141, 'moo': '0', 'room': None, 'alley'...",มีนบุรี,True
4,นาย มาโนชญ์ ลีโทชวลิต,สังข์กระจายคลินิกการแพทย์,บ้านเลขที่849/4 ซอยวัดสังข์กระจาย ถนนเพชรเก...,849/4,NaN,NaN,NaN,วัดสังข์กระจาย,เพชรเกษม,NaN,วัดท่าพระ,บางกอกใหญ่,กรุงเทพมหานคร,10600.0,0 2466 8564,NaN,นาย มาโนชญ์ ลีโทชวลิต,ครอบครอง วจ4 สถานพยาบาล เอกชน,5900667.0,22/2560 (วจ.4),อนุมัติ,NaN,สังข์กระจายคลินิกการแพทย์,CA0024201,"{'id': 6007, 'moo': '0', 'room': null, 'alley'...",both,"{'id': 6007, 'moo': '0', 'room': None, 'alley'...",บางกอกใหญ่,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,บริษัท นวศรี การแพทย์ จำกัด,โรงพยาบาลผู้ป่วยเรื้อรังศุขเวชขนาดเล็ก,บ้านเลขที่20/6 ซอยนวศรี 2 ถนนรามคำแหง 21 หม...,20/6,NaN,NaN,NaN,นวศรี 2,รามคำแหง 21,NaN,พลับพลา,วังทองหลาง,กรุงเทพมหานคร,10310.0,0 2319 5870-1,0 2319 5871,นาย มานิตย์ ประพันธ์ศิลป์,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6700649.0,121/2567,อนุมัติ,NaN,โรงพยาบาลผู้ป่วยเรื้อรังศุขเวชขนาดเล็ก,HA0014578,"{'id': 2475, 'moo': '0', 'room': '', 'alley': ...",both,"{'id': 2475, 'moo': '0', 'room': '', 'alley': ...",วังทองหลาง,True
267,บริษัท เป็นเลิศเมดิคอล จำกัด,สิทธิรักษ์คลินิกเวชกรรม,"บ้านเลขที่128,130 ชั้น 1 ถนนเลียบวารี ตำบล...","128,130",NaN,ชั้น 1,NaN,NaN,เลียบวารี,NaN,กระทุ่มราย,หนองจอก,กรุงเทพมหานคร,10530.0,0 2100 4542,NaN,นาย นันทศักดิ์ ธรรมานวัตร์,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800017.0,ควจ4-ร 6/2568,อนุมัติ,NaN,สิทธิรักษ์คลินิกเวชกรรม,CA0048975,"{'id': 35172, 'moo': '', 'room': '', 'alley': ...",both,"{'id': 35172, 'moo': '', 'room': '', 'alley': ...",หนองจอก,True
268,มูลนิธิ ฮั่วเคี้ยวป่อเต็กเซี่ยงตึ๊ง(ป่อเต็กตึ๊ง),โรงพยาบาลหัวเฉียวโรงพยาบาลทั่วไปขนาดใหญ่,บ้านเลขที่665 ซอย ถนนบำรุงเมือง หมู่ ตำบลคล...,665,NaN,NaN,NaN,NaN,บำรุงเมือง,NaN,คลองมหานาค,ป้อมปราบศัตรูพ่าย,กรุงเทพมหานคร,10100.0,0 2223 1351 ต่อ 5507-8,0 2223 1253,นาง มนนภา ขุนณรงค์,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800030.0,ควจ4-ร 9/2568,อนุมัติ,NaN,โรงพยาบาลหัวเฉียวโรงพยาบาลทั่วไปขนาดใหญ่,HA0011750,"{'id': 13186, 'moo': '0', '

In [216]:
merge2['district'].eq(merge2['อำเภอ'])

0      False
1      False
2       True
3       True
4       True
       ...  
266     True
267     True
268     True
269     True
270     True
Length: 271, dtype: bool

In [213]:
merge2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   ชื่อผู้รับอนุญาต     271 non-null    object  
 1   ชื่อสถานที่          271 non-null    object  
 2   ที่อยู่สถานที่       271 non-null    object  
 3   บ้านเลขที่           271 non-null    object  
 4   ห้อง                 14 non-null     object  
 5   ชั้น                 65 non-null     object  
 6   อาคาร                25 non-null     object  
 7   ซอย                  108 non-null    object  
 8   ถนน                  224 non-null    object  
 9   หมู่                 14 non-null     object  
 10  ตำบล                 271 non-null    object  
 11  อำเภอ                271 non-null    object  
 12  จังหวัด              271 non-null    object  
 13  รหสไปรษณีย์          271 non-null    float64 
 14  เบอร์โทรศัพท์        266 non-null    object  
 15  เบอร์โทรสาร          58

In [234]:
parsed_address(hcode_csv[hcode_csv['name'] == 'คลินิกอุทัยการแพทย์']['address'].iloc[0])

{'id': 10945,
 'moo': '0',
 'room': None,
 'alley': None,
 'floor': None,
 'steet': None,
 'number': None,
 'region': 'ภาคใต้',
 'details': '100 ถ.จุติอนุสรณ์\r\n',
 'village': None,
 'zipcode': '90110',
 'building': None,
 'district': 'หาดใหญ่',
 'province': 'สงขลา',
 'health_zone': '12',
 'subdistrict': 'หาดใหญ่',
 'district_code': '9011',
 'province_code': '90',
 'province_type': 'จังหวัด',
 'is_double_space': False,
 'subdistrict_code': '901101'}

In [230]:
miss_matched_district = merge2.loc[merge2['district'].eq(merge2['อำเภอ']) ==False].copy()

In [272]:
miss_matched_district.iloc[10]

ชื่อผู้รับอนุญาต                               นาย ประเสริฐ ศรีทองส่งแสง
ชื่อสถานที่                                        ประเสริฐคลินิกเวชกรรม
ที่อยู่สถานที่         บ้านเลขที่160/4    ซอย ถนนวิสุทธิ์กษัตริย์ หมู...
บ้านเลขที่                                                         160/4
ห้อง                                                                 NaN
ชั้น                                                                 NaN
อาคาร                                                                NaN
ซอย                                                                  NaN
ถนน                                                     วิสุทธิ์กษัตริย์
หมู่                                                                 NaN
ตำบล                                                           บ้านพานถม
อำเภอ                                                             พระนคร
จังหวัด                                                    กรุงเทพมหานคร
รหสไปรษณีย์                                        

In [311]:
hcode_csv[hcode_csv['name'] == 'พี.เอส.คลินิกเวชกรรม']

,#,id,name,hcode9,code9,code5,private_code,organization_type,health_office_type,health_office_type_code,organization,organization_code,department,department_code,active,address,established_date,closed_date,bed,is_client_hospital,host_agency,client_health_office_type,modified_date,notes,parsed_address,formatted_address,location_type,plcae_id,lat,lng,subtypes,pv_code,pv,obj_address,district
14804,14805,14806,พี.เอส.คลินิกเวชกรรม,CA0040267,4026700.0,40267.0,11101003555,เอกชน,คลินิกเอกชน,16,เอกชน,90000,NaN,NaN,True,"{'id': 14806, 'moo': '2', 'room': null, 'alley...",NaN,NaN,NaN,False,NaN,NaN,2023-08-31T13:33:12.723668+07:00,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,10270,10,"{'id': 14806, 'moo': '2', 'room': None, 'alley...",เมืองสมุทรปราการ


In [313]:
#correct 1:m case
remap=[{"iloc":7,
        "hcode9":"CA0024491"},
       {"iloc":17,
        "hcode9":"CA0022525"},
]

In [312]:
miss_matched_district['name'].iloc[27]

IndexError: single positional indexer is out-of-bounds

In [302]:
miss_matched_district.iloc[23]

ชื่อผู้รับอนุญาต                               บริษัท ชีวีบริรักษ์ จำกัด
ชื่อสถานที่                                    ชีวีบริรักษ์คลินิกเวชกรรม
ที่อยู่สถานที่         บ้านเลขที่86/5  ชั้น 2-3  ซอยสุขุมวิท 89   ตำบ...
บ้านเลขที่                                                          86/5
ห้อง                                                                 NaN
ชั้น                                                            ชั้น 2-3
อาคาร                                                                NaN
ซอย                                                          สุขุมวิท 89
ถนน                                                                  NaN
หมู่                                                                 NaN
ตำบล                                                              บางจาก
อำเภอ                                                            พระโขนง
จังหวัด                                                    กรุงเทพมหานคร
รหสไปรษณีย์                                        

In [254]:
hcode_csv['obj_address'] = hcode_csv['address'].apply(parsed_address)

In [255]:
hcode_csv.loc[hcode_csv['obj_address'].isna()]

,#,id,name,hcode9,code9,code5,private_code,organization_type,health_office_type,health_office_type_code,organization,organization_code,department,department_code,active,address,established_date,closed_date,bed,is_client_hospital,host_agency,client_health_office_type,modified_date,notes,parsed_address,formatted_address,location_type,plcae_id,lat,lng,subtypes,pv_code,pv,obj_address


In [256]:
hcode_csv['district'] = hcode_csv['obj_address'].apply(get_dsitrict)